# Introduccion

Este notebook contiene los procesos que se han realizado para cargar y limpiar las bases de datos de contratos de la **Agencia Nacional de Contratacion Pública - Colombia COmpra Eficiente** (ANCP-CCE) en sus sistemas SECOP-I y SECOP-II, accesibles  a trvés de la plataforma Datos Abiertos.

Tambien contiene las funciones e scripts para cargar los datasets en Jupyter o Colab, a partir de archivos en formato CSV.

Esta es la primera etapa del trabajo con esstos datos, que consiste en conslidarlos en un solo archivo asegurando su consistencia y relevancia para el trabajo de la Superintendencia de Vigilancia y Seguridad Privada.


In [3]:
# First import the required libraries.
import io
import os
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns

In [4]:
# Ajustar No Maximo de Columnas para mostrar
# pd.get_option("display.max_columns")
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_columns', 20) # reset to default

In [5]:
# Ajustar el tamaño máximo de columna que se muestra
#pd.get_option('display.max_colwidth')
#pd.set_option('display.max_colwidth', -1) 
#pd.set_option('display.max_colwidth', 50)  # reset to default

In [6]:
# Ajustar el número máximo de filas para mostrar
#pd.get_option('display.max_rows')
pd.set_option('display.max_rows', None) # mostrar todas
#pd.set_option('display.max_rows', 60) # reset to default

In [7]:
# Ajustar la forma en que se muestran los números (NOtación científica vs contable)
pd.set_option('display.float_format',  '{:,.2f}'.format)

# Step 1: **Data Loading**

## Load CSV into colab from Google Drive (DO NOT RUN if in Jupyter):

In [8]:
# For Using Google Drive, (Only if executing notebook from Google colab):
from google.colab import drive
drive.mount('/content/drive')

# after that:
# <--- Refresh mounted Drive
# <--- Look for file and get the path link

Mounted at /content/drive


### Find the file

In [9]:
# Get the current directory
print(os.getcwd())
# Check if the directory exists
print(os.path.exists('../'))

/content
True


In [10]:
ls

drive/  sample_data/


In [11]:
cd /content/drive/MyDrive/Observatorio SPVS/

/content/drive/MyDrive/Observatorio SPVS


In [12]:
# To load a file into colab:

### Tabla Contratos de Vigilancia en SECOP

In [11]:
PATH = '/content/drive/MyDrive/Observatorio SPVS/BD SECOP-II - Contratos/SECOP-II_-_Procesos_de_Contratacion_Filtro_Vigilancia_Datos Validos.csv'
df_SECOP = pd.read_csv(PATH, encoding = 'utf-8', dtype='str', sep=',' )
df_SECOP.shape

(752, 60)

In [12]:
columnas_utiles = df_SECOP.columns

# Define la lista de columnas que se van a necesitar
columnas_utiles = ['Entidad', 'Nit Entidad', 'Departamento Entidad', 'Ciudad Entidad',
       'OrdenEntidad', 'Entidad Centralizada', 'ID del Proceso',
       'Referencia del Proceso',
       'Precio Base', 'Modalidad de Contratacion', 'Justificación Modalidad de Contratación',
       'Duracion en Días',
       'Ciudad de la Unidad de Contratación',
       'Nombre de la Unidad de Contratación', 
       'ID Adjudicacion', 'CodigoProveedor', 'Departamento Proveedor',
       'Ciudad Proveedor', 'Fecha Adjudicacion', 'Valor Total Adjudicacion',
       'Nombre del Adjudicador', 'Nombre del Proveedor Adjudicado',
       'NIT del Proveedor Adjudicado', 'Codigo Principal de Categoria',
       'Estado de Apertura del Proceso', 'Tipo de Contrato',
       'Categorias Adicionales', 'URLProceso',
       'Codigo Entidad']

In [13]:
df_SECOP = df_SECOP[columnas_utiles]
df_SECOP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752 entries, 0 to 751
Data columns (total 29 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   Entidad                                  752 non-null    object
 1   Nit Entidad                              752 non-null    object
 2   Departamento Entidad                     752 non-null    object
 3   Ciudad Entidad                           752 non-null    object
 4   OrdenEntidad                             752 non-null    object
 5   Entidad Centralizada                     752 non-null    object
 6   ID del Proceso                           752 non-null    object
 7   Referencia del Proceso                   752 non-null    object
 8   Precio Base                              752 non-null    object
 9   Modalidad de Contratacion                752 non-null    object
 10  Justificación Modalidad de Contratación  752 non-null    objec

In [14]:
df_SECOP['Entidad'].unique

<bound method Series.unique of 0                       CGR GERENCIA DEPARTAMENTAL HUILA
1              SENA DIRECCIÓN GENERAL Dirección Jurídica
2                   SECRETARIA DE EDUCACION DEL DISTRITO
3      AGENCIA NACIONAL DE CONTRATACIÓN PÚBLICA -COLO...
4          INSTITUTO DE TRANSPORTES Y TRANSITO DEL HUILA
5                                FUERZA AÉREA COLOMBIANA
6             AGENCIA LOGISTICA DE LAS FUERZAS MILITARES
7                                  DEPARTAMENTO DEL META
8      INSTITUTO NACIONAL PENITENCIARIO Y CARCELARIO ...
9                                 MUNICIPIO DE SINCELEJO
10                              MUNICIPIO DE BUCARAMANGA
11                            INSTITUTO NACIONAL DE VIAS
12     PARQUES NACIONALES NATURALES DE COLOMBIA - DIR...
13        INSTITUTO DE CULTURA Y BELLAS ARTES DE DUITAMA
14     FONDO UNICO DE TECNOLOGÍAS DE LA INFORMACIÓN Y...
15                        INSTITUTO DE DESARROLLO URBANO
16                               FUERZA AÉREA COLOMBIANA


In [16]:
df_SECOP['Estado del Procedimiento'].unique()

KeyError: ignored

In [17]:
df_SECOP['Adjudicado'].unique()

KeyError: ignored

In [ ]:
df_adjudicado = df.loc[df['Adjudicado']=='Si']
df_adjudicado = pd.concat([df_adjudicado, df.loc[df['Estado del Procedimiento']=='Adjudicado']])
df_adjudicado.drop_duplicates(inplace = True)
df_adjudicado.shape

## Códigos de Actividad UNSPSC

https://www.colombiacompra.gov.co/sites/cce_public/files/cce_clasificador/unspsc_spanish_v14_0801.pdf



In [18]:
# Diccionario con los códigos UNSPSC de actividades y servicios relevantes para el sector de vigilancia y seguridad privada
UNSPSC_servicios_codes = pd.read_excel('/content/drive/MyDrive/Observatorio SPVS/BD SECOP-II - Contratos/clasificacion UNSPSC.xlsx',
                                       sheet_name='Servicios de Vigilancia') #dtype={value.dtype} # , dtype='str', sep=',' 
#UNSPSC_servicios_codes

In [19]:
# Dataframe con los códigos UNSPSC relevantes para el sector de vigilancia y seguridad privada
UNSPSC_equipos_codes = pd.read_excel('/content/drive/MyDrive/Observatorio SPVS/BD SECOP-II - Contratos/clasificacion UNSPSC.xlsx',
                                   sheet_name='Eq. Vig. y Seguridad') #dtype={value.dtype} # , dtype='str', sep=',' 
#UNSPSC_equipos_codes

In [20]:
# Dataframe con los códigos UNSPSC relevantes para el sector de vigilancia y seguridad privada
UNSPSC_blindados_codes = pd.read_excel('/content/drive/MyDrive/Observatorio SPVS/BD SECOP-II - Contratos/clasificacion UNSPSC.xlsx',
                                   sheet_name='Blindados') #dtype={value.dtype} # , dtype='str', sep=',' 
#UNSPSC_blindados_codes

In [21]:
# Dataframe con los códigos UNSPSC de armas
UNSPSC_armas_codes = pd.read_excel('/content/drive/MyDrive/Observatorio SPVS/BD SECOP-II - Contratos/clasificacion UNSPSC.xlsx',
                                     sheet_name='Armas') #dtype={value.dtype} # , dtype='str', sep=',' 
#UNSPSC_armas_codes

In [22]:
# Dataframe con los códigos UNSPSC relevantes para el sector de vigilancia y seguridad privada
UNSPSC_otros_codes = pd.read_excel('/content/drive/MyDrive/Observatorio SPVS/BD SECOP-II - Contratos/clasificacion UNSPSC.xlsx',
                                   sheet_name='Serv. Apoyo') #dtype={value.dtype} # , dtype='str', sep=',' 
#UNSPSC_otros_codes

In [23]:
UNSPSC_codes = pd.concat([UNSPSC_servicios_codes, UNSPSC_equipos_codes, UNSPSC_blindados_codes, UNSPSC_armas_codes, UNSPSC_otros_codes]) #.reset_index(drop=True)
UNSPSC_codes.set_index('UNSPSC', drop = True, inplace = True)
#UNSPSC_codes

In [24]:
UNSPSC_codes_dict = {
'Servicio de Vigilancia y Seguridad Privada' : [ 
                'V1.92101501', # Servicios de vigilancia
                'V1.9212', # Seguridad y protección personal
                'V1.921215', # Servicios de guardias
                'V1.921217', # Servicios de sistemas de seguridad 
                'V1.93121503' # Servicios de seguridad para diplomáticos 
                ],
'Servicio de Asesoria, Consultoria e Investigacion' : [
                'V1.80111702', # Servicios de comprobación de referencias o antecedentes
                'V1.81141801', # Análisis de riesgo o seguridad
                'V1.921216', # Servicios de detectives
                ],
'Servicio de Capacitacion y Entrenamiento en Vigilancia y Seguridad Privada' : [
                'V1.86101709', # Servicios de capacitación en seguridad
                ],
'Fabricacion, Importacion, Comercio, Arriendo e Instalación de Equipos para la Vigilancia y la Seguridad Privada' : [ 
                'V1.461516', # Equipo de seguridad y control
                'V1.461518', # Equipo y accesorios y suministros de control de explosivos
                #'V1.461519', # Equipo y accesorios y suministros para el control químico biológico
                'V1.4617', # Seguridad, vigilancia y detección
                'V1.461715', # Cerraduras y elementos de seguridad y accesorios
                'V1.461716', # Equipo de vigilancia y detección
                'V1.721517', # Servicios de instalación de sistemas de seguridad física e industrial
               ],
'Servicio de Transporte de Valores' : [
                'V1.78101503', # Transporte aéreo blindado
                'V1.78101705', # Transporte marítimo blindado
                'V1.921218', # Servicios de Carros blindados y transporte de valores
                 #'V1.921219' # Servicios de seguridad marina
               ],
'Servicio de Fabricación y/o Arrendamiento de Vehículos Blindados' : [ 
                'V1.25101920' # Carros de Seguridad, Vehículos Comerciales, Militares y Particulares, Accesorios y Componentes
               ],
'Suministro de Armas': [
          'V1.460000', # Equipos y Suministros de Defensa, Orden Publico, Proteccion, Vigilancia y Seguridad
          'V1.4610', # Armas ligeras y munición
          'V1.461015', # Armas de fuego
          'V1.461016', # Munición
          'V1.461017', # Sistemas de manipulación de munición
          'V1.461018' # Accesorios de armas y municiones
          ]
}

# Se deben quitar 461519 y 921219 porque al revisar los contratos en detalle, se encontró que no corresponden con el sector, son insumos químicos y aseguramiento financiero


In [25]:
for categoria in UNSPSC_codes_dict:
    print (categoria, UNSPSC_codes_dict[categoria])

Servicio de Vigilancia y Seguridad Privada ['V1.92101501', 'V1.9212', 'V1.921215', 'V1.921217', 'V1.93121503']
Servicio de Asesoria, Consultoria e Investigacion ['V1.80111702', 'V1.81141801', 'V1.921216']
Servicio de Capacitacion y Entrenamiento en Vigilancia y Seguridad Privada ['V1.86101709']
Fabricacion, Importacion, Comercio, Arriendo e Instalación de Equipos para la Vigilancia y la Seguridad Privada ['V1.461516', 'V1.461518', 'V1.4617', 'V1.461715', 'V1.461716', 'V1.721517']
Servicio de Transporte de Valores ['V1.78101503', 'V1.78101705', 'V1.921218']
Servicio de Fabricación y/o Arrendamiento de Vehículos Blindados ['V1.25101920']
Suministro de Armas ['V1.460000', 'V1.4610', 'V1.461015', 'V1.461016', 'V1.461017', 'V1.461018']


In [26]:
for categoria, lista in UNSPSC_codes_dict.items():
    print (categoria)
    for unspsc in lista:
         print(unspsc)

Servicio de Vigilancia y Seguridad Privada
V1.92101501
V1.9212
V1.921215
V1.921217
V1.93121503
Servicio de Asesoria, Consultoria e Investigacion
V1.80111702
V1.81141801
V1.921216
Servicio de Capacitacion y Entrenamiento en Vigilancia y Seguridad Privada
V1.86101709
Fabricacion, Importacion, Comercio, Arriendo e Instalación de Equipos para la Vigilancia y la Seguridad Privada
V1.461516
V1.461518
V1.4617
V1.461715
V1.461716
V1.721517
Servicio de Transporte de Valores
V1.78101503
V1.78101705
V1.921218
Servicio de Fabricación y/o Arrendamiento de Vehículos Blindados
V1.25101920
Suministro de Armas
V1.460000
V1.4610
V1.461015
V1.461016
V1.461017
V1.461018


## Conexión con SOCRATA (datos.gov.co)


In [27]:
pip install sodapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
from sodapy import Socrata

In [29]:
client = Socrata("www.datos.gov.co",
                 "TJ4yjQcX9XrhC8MB4EBqnYISj",## Cambie token_aqui por el token recien creado
                 username="ma.garzon.ra@gmail.com", ## Correo electrónico de ingreso a la plataforma de datos abiertos
                 password="Papas1597")## Contraseña de ingreso a la plataforma de datos abiertos


### TESTS

In [30]:
#Test SECOP - I
%%time
socrata_dataset_identifier = "f789-7hwg"

Query = """
SELECT * WHERE    fecha_de_firma_del_contrato >= '2018-01-01'
LIMIT 100
"""

contratos_SI = client.get(socrata_dataset_identifier, content_type="json", query=Query)

contratos_SI_DF = pd.DataFrame(pd.DataFrame.from_dict(contratos_SI))

CPU times: user 42.2 ms, sys: 5.28 ms, total: 47.5 ms
Wall time: 2.65 s


In [31]:
# Test  SECOP - II
%%time
socrata_dataset_identifier = "jbjy-vk9h"

Query = """
SELECT * WHERE    fecha_de_firma >= '2018-01-01'
LIMIT 100
"""

contratos_SII = client.get(socrata_dataset_identifier, content_type="json", query=Query)

contratos_SII_df = pd.DataFrame(pd.DataFrame.from_dict(contratos_SII))

CPU times: user 17.3 ms, sys: 1.12 ms, total: 18.4 ms
Wall time: 557 ms


In [32]:
contratos_SI_DF.head(3)

,uid,anno_cargue_secop,anno_firma_contrato,nivel_entidad,orden_entidad,nombre_entidad,nit_de_la_entidad,c_digo_de_la_entidad,id_modalidad,modalidad_de_contratacion,estado_del_proceso,causal_de_otras_formas_de,id_regimen_de_contratacion,nombre_regimen_de_contratacion,id_objeto_a_contratar,objeto_a_contratar,detalle_del_objeto_a_contratar,tipo_de_contrato,municipio_de_obtencion,municipio_de_entrega,municipios_ejecucion,fecha_de_cargue_en_el_secop,numero_de_constancia,numero_de_proceso,numero_de_contrato,cuantia_proceso,id_grupo,nombre_grupo,id_familia,nombre_familia,id_clase,nombre_clase,id_adjudicacion,tipo_identifi_del_contratista,identificacion_del_contratista,nom_razon_social_contratista,dpto_y_muni_contratista,tipo_doc_representante_legal,identific_representante_legal,nombre_del_represen_legal,fecha_de_firma_del_contrato,fecha_ini_ejec_contrato,plazo_de_ejec_del_contrato,rango_de_ejec_del_contrato,tiempo_adiciones_en_dias,tiempo_adiciones_en_meses,fecha_fin_ejec_contrato,compromiso_presupuestal,cuantia_contrato,valor_total_de_adiciones,valor_contrato_con_adiciones,objeto_del_contrato_a_la,proponentes_seleccionados,calificacion_definitiva,id_sub_unidad_ejecutora,nombre_sub_unidad_ejecutora,ruta_proceso_en_secop_i,moneda,es_postconflicto,marcacion_adiciones,posicion_rubro,nombre_rubro,valor_rubro,sexo_replegal,pilar_acuerdo_paz,punto_acuerdo_paz,municipio_entidad,departamento_entidad,ultima_actualizacion,cumpledecreto248,incluyebienesdecreto248,fecha_liquidacion
0,18-12-7863192-7138877,2018,1905,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,META HOSPITAL DE CASTILLA LA NUEVA ESE,900004059,250150017,12,Contratación Directa (Ley 1150 de 2007),Celebrado,Prestación de Servicios Profesionales y de Apo...,12,Contratación Directa (Ley 1150 de 2007),85000000,Servicios de Salud,PRESTACION DE SERVICIOS COMO PROFESIONAL EN BA...,Prestación de Servicios,Castilla La Nueva,Castilla La Nueva,"Castilla La Nueva, Meta",2018-02-13T00:00:00.000,18-12-7863192,2018059,2018059,14700000,F,[F] Servicios,8510,Servicios integrales de salud,851016,Personas de soporte de prestación de servicios...,7138877,Cédula de Ciudadanía,60376429,EDNA DEICE HURTADO PEREZ,Meta,Cédula de Ciudadanía,60376429,EDNA DEICE HURTADO PEREZ,2018-01-01T00:00:00.000,2018-01-01T00:00:00.000,4,M,0,2,2018-07-01T00:00:00.000,Sn Definir,14700000,7350000,22050000,PRESTACION DE SERVICIOS COMO PROFESIONAL EN BA...,No Definido,No Definido,0,No Definido,{'url': 'https://www.contratos.gov.co/consulta...,Peso Colombiano,0,0,No Definido,No Definido,0,N,No Definido,No Definido,Castilla La Nueva,Meta,2018-02-13T00:00:00.000,N/D,N/D,NaN
1,18-12-7533198-6845825,2018,1905,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,RISARALDA ESE SALUD PEREIRA,816005003,266001463,12,Contratación Directa (Ley 1150 de 2007),Celebrado,Prestación de Servicios Profesionales y de Apo...,12,Contratación Directa (Ley 1150 de 2007),85000000,Servicios de Salud,PRESTACION DE SERVICIOS PROFESIONALES COMO MED...,Prestación de Servicios,No Definido,No Definido,"Pereira, Risaralda",2018-01-15T00:00:00.000,18-12-7533198,5918,5918,36400000,F,[F] Servicios,8510,Servicios integrales de salud,851016,Personas de soporte de prestación de servicios...,6845825,Cédula de Ciudadanía,1144133036,VIVIAN ROCIO CORDOBA ALMARIO,Risaralda,Cédula de Ciudadanía,1144133036,VIVIAN ROCIO CORDOBA ALMARIO,2018-01-01T00:00:00.000,2018-01-01T00:00:00.000,7,M,0,0,2018-08-01T00:00:00.000,Sn Definir,36400000,0,36400000,PRESTACION DE SERVICIOS PROFESIONALES COMO MED...,No Definido,No Definido,0,No Definido,{'url': 'https://www.contratos.gov.co/consulta...,Peso Colombiano,0,0,No Definido,No Definido,0,N,No Definido,No Definido,Pereira,Risaralda,2018-01-15T00:00:00.000,N/D,N/D,NaN
2,18-12-7493899-6810252,2018,1905,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,TOLIMA UNIDAD DE SALUD DE IBAGUÉ,809003590,26617044,12,Contratación Directa (Ley 1150 de 2007),Celebrado,Prestación de Servicios Profesionales y de Apo...,12,Contratación Directa

In [33]:
contratos_SII_df.head(3)

,nombre_entidad,nit_entidad,departamento,ciudad,localizaci_n,orden,sector,rama,entidad_centralizada,proceso_de_compra,id_contrato,referencia_del_contrato,estado_contrato,codigo_de_categoria_principal,descripcion_del_proceso,tipo_de_contrato,modalidad_de_contratacion,justificacion_modalidad_de,fecha_de_firma,fecha_de_inicio_del_contrato,fecha_de_fin_del_contrato,condiciones_de_entrega,tipodocproveedor,documento_proveedor,proveedor_adjudicado,es_grupo,es_pyme,habilita_pago_adelantado,liquidaci_n,obligaci_n_ambiental,obligaciones_postconsumo,reversion,valor_del_contrato,valor_de_pago_adelantado,valor_facturado,valor_pendiente_de_pago,valor_pagado,valor_amortizado,valor_pendiente_de,valor_pendiente_de_ejecucion,estado_bpin,c_digo_bpin,anno_bpin,saldo_cdp,saldo_vigencia,espostconflicto,urlproceso,destino_gasto,origen_de_los_recursos,dias_adicionados,puntos_del_acuerdo,pilares_del_acuerdo,nombre_representante_legal,nacionalidad_representante_legal,tipo_de_identificaci_n_representante_legal,identificaci_n_representante_legal,g_nero_representante_legal,presupuesto_general_de_la_nacion_pgn,sistema_general_de_participaciones,sistema_general_de_regal_as,recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_,recursos_de_credito,recursos_propios,ultima_actualizacion,codigo_entidad,codigo_proveedor,objeto_del_contrato,fecha_de_inicio_de_ejecucion,fecha_de_fin_de_ejecucion,fecha_inicio_liquidacion,fecha_fin_liquidacion
0,JURISDICCION ESPECIAL DE PAZ,901140004,Distrito Capital de Bogotá,Bogotá,"Colombia, Bogotá, Bogotá",Nacional,No aplica/No pertenece,Corporación Autónoma,Centralizada,CO1.BDOS.745128,CO1.PCCNTR.835071,JEP - 28 -2019,Activo,V1.93141500,PRESTACIÓN DE SERVICIOS PROFESIONALES ESPECIA...,Prestación de servicios,Contratación directa,ServiciosProfesionales,2019-02-18T08:02:03.000,2019-02-01T00:00:00.000,2019-12-31T00:00:00.000,Como acordado previamente,NIT,79261868,Jose Arles Nova Villanueva,No,No,No,No,No,No,No,114393961,0,0,114393961,0,0,0,114393961,Válido,No Definido,2019,0,0,Si,{'url': 'https://community.secop.gov.co/Public...,Inversión,Distribuido,0,No aplica,No aplica,José Arles Nova Villanueva,Colombiano,No Definido,79261868,No Definido,0,0,0,0,0,0,2022-02-23T00:00:00.000,703840058,706097326,PRESTACIÓN DE SERVICIOS PROFESIONALES ESPECIAL...,NaN,NaN,NaN,NaN
1,Secretaria Distrital de Integración Social - O...,8999990611,Distrito Capital de Bogotá,No Definido,"Colombia, Bogotá, Bogotá",Territorial,Inclusión Social y Reconciliación,Ejecutivo,Centralizada,CO1.BDOS.1241507,CO1.PCCNTR.1571334,4552-2020,terminado,V1.80111500,PRESTAR SERVICIOS PARA LA ATENCIÓN INTEGRAL A ...,Prestación de servicios,Contratación directa,ServiciosProfesionales,2020-05-18T11:05:14.000,2020-06-01T00:00:00.000,2020-10-30T00:00:00.000,No Definido,Cédula de Ciudadanía,52548024,JIMENA VERGARA GARNICA,No,No,No,No,No,No,No,16390000,0,15297333,1092667,15297333,0,0,16390000,Válido,No Definido,2020,16390000,0,No,{'url': 'https://community.secop.gov.co/Public...,Inversión,Distribuido,0,No aplica,No aplica,JIMENA VERGARA GARNICA,COLOMBIANA,No Definido,52548024,No Definido,0,0,0,16390000,0,0,2020-11-03T00:00:00.000,702271321,705386662,PRESTAR SERVICIOS PARA LA ATENCIÓN INTEGRAL A ...,NaN,NaN,NaN,NaN
2,SERVICIO GEOLOGICO COLOMBIANO,899999294,Distrito Capital de Bogotá,Bogotá,"Colombia, Bogotá, Bogotá",Nacional,Minas y Energía,Ejecutivo,Centralizada,CO1.BDOS.638101,CO1.PCCNTR.696402,CONTRATO 885 DE 2018,Activo,V1.81151901,El Contratista se compromete para con el Servi...,Prestación de servicios,Contratación directa,ServiciosProfesionales,2018-12-19T21:12:01.000,2018-12-19T00:00:00.000,2019-08-31T00:00:00.000,A convenir,Cédula de Ciudadanía,12749676,Dario Fernando Arcos Guerrero,No,No,No,No,No,No,No,59259607,0,0,59259607,0,0,0,59259607,Válido,No Definido,N/D,0,0,No,{'url': 'https://community.secop.gov.co/Public...,No Definido,Distribuido,0,No aplica,No aplica,Dario Fernando Arcos Guerrero,Colombiano,No Definido,12749676,No Definido,0,0,0,0,0,0,2022-03-25T00:0

In [34]:
contratos_SII_df.columns

Index(['nombre_entidad', 'nit_entidad', 'departamento', 'ciudad',
       'localizaci_n', 'orden', 'sector', 'rama', 'entidad_centralizada',
       'proceso_de_compra', 'id_contrato', 'referencia_del_contrato',
       'estado_contrato', 'codigo_de_categoria_principal',
       'descripcion_del_proceso', 'tipo_de_contrato',
       'modalidad_de_contratacion', 'justificacion_modalidad_de',
       'fecha_de_firma', 'fecha_de_inicio_del_contrato',
       'fecha_de_fin_del_contrato', 'condiciones_de_entrega',
       'tipodocproveedor', 'documento_proveedor', 'proveedor_adjudicado',
       'es_grupo', 'es_pyme', 'habilita_pago_adelantado', 'liquidaci_n',
       'obligaci_n_ambiental', 'obligaciones_postconsumo', 'reversion',
       'valor_del_contrato', 'valor_de_pago_adelantado', 'valor_facturado',
       'valor_pendiente_de_pago', 'valor_pagado', 'valor_amortizado',
       'valor_pendiente_de', 'valor_pendiente_de_ejecucion', 'estado_bpin',
       'c_digo_bpin', 'anno_bpin', 'saldo_cdp', 'sa

In [35]:
# Columnas disponibles en el dataset SECOP - I
contratos_SI_DF.columns
len(contratos_SI_DF.columns)
# Todas las columnas, para copiar en el query
""" uid,anno_cargue_secop,anno_firma_contrato,nivel_entidad,orden_entidad,nombre_entidad,nit_de_la_entidad,
c_digo_de_la_entidad,id_modalidad,modalidad_de_contratacion,estado_del_proceso,causal_de_otras_formas_de,
id_regimen_de_contratacion,nombre_regimen_de_contratacion,id_objeto_a_contratar,objeto_a_contratar,
detalle_del_objeto_a_contratar,tipo_de_contrato,municipio_de_obtencion,municipio_de_entrega,municipios_ejecucion,
fecha_de_cargue_en_el_secop,numero_de_constancia,numero_de_proceso,numero_de_contrato,cuantia_proceso,id_grupo,nombre_grupo,
id_familia,nombre_familia,id_clase,nombre_clase,id_adjudicacion,tipo_identifi_del_contratista,identificacion_del_contratista,
nom_razon_social_contratista,dpto_y_muni_contratista,tipo_doc_representante_legal,identific_representante_legal,
nombre_del_represen_legal,fecha_de_firma_del_contrato,fecha_ini_ejec_contrato,plazo_de_ejec_del_contrato,
rango_de_ejec_del_contrato,tiempo_adiciones_en_dias,tiempo_adiciones_en_meses,fecha_fin_ejec_contrato,
compromiso_presupuestal,cuantia_contrato,valor_total_de_adiciones,valor_contrato_con_adiciones,objeto_del_contrato_a_la,
proponentes_seleccionados,calificacion_definitiva,id_sub_unidad_ejecutora,nombre_sub_unidad_ejecutora,
ruta_proceso_en_secop_i,moneda,es_postconflicto,marcacion_adiciones,posicion_rubro,nombre_rubro,valor_rubro,
sexo_replegal,pilar_acuerdo_paz,punto_acuerdo_paz,municipio_entidad,departamento_entidad,ultima_actualizacion,fecha_liquidacion,"""

' uid,anno_cargue_secop,anno_firma_contrato,nivel_entidad,orden_entidad,nombre_entidad,nit_de_la_entidad,\nc_digo_de_la_entidad,id_modalidad,modalidad_de_contratacion,estado_del_proceso,causal_de_otras_formas_de,\nid_regimen_de_contratacion,nombre_regimen_de_contratacion,id_objeto_a_contratar,objeto_a_contratar,\ndetalle_del_objeto_a_contratar,tipo_de_contrato,municipio_de_obtencion,municipio_de_entrega,municipios_ejecucion,\nfecha_de_cargue_en_el_secop,numero_de_constancia,numero_de_proceso,numero_de_contrato,cuantia_proceso,id_grupo,nombre_grupo,\nid_familia,nombre_familia,id_clase,nombre_clase,id_adjudicacion,tipo_identifi_del_contratista,identificacion_del_contratista,\nnom_razon_social_contratista,dpto_y_muni_contratista,tipo_doc_representante_legal,identific_representante_legal,\nnombre_del_represen_legal,fecha_de_firma_del_contrato,fecha_ini_ejec_contrato,plazo_de_ejec_del_contrato,\nrango_de_ejec_del_contrato,tiempo_adiciones_en_dias,tiempo_adiciones_en_meses,fecha_fin_ejec_con

In [36]:
columnas_SI = ['uid',
'numero_de_constancia',
'numero_de_proceso',
'numero_de_contrato',
'anno_firma_contrato',
'anno_cargue_secop',
'nivel_entidad',
'orden_entidad',
'nombre_entidad',
'nit_de_la_entidad',
'c_digo_de_la_entidad',
'modalidad_de_contratacion',
'estado_del_proceso',
'objeto_a_contratar',
'objeto_del_contrato_a_la',
'detalle_del_objeto_a_contratar',
'tipo_de_contrato',
'municipio_entidad',
'departamento_entidad',
'id_grupo',
'nombre_grupo',
'id_familia',
'nombre_familia',
'id_clase',
'id_clase',
'nombre_clase',
'tipo_identifi_del_contratista',
'identificacion_del_contratista',
'nom_razon_social_contratista',
'tipo_doc_representante_legal',
'identific_representante_legal',
'nombre_del_represen_legal',
'sexo_replegal',
'fecha_de_firma_del_contrato',
'fecha_ini_ejec_contrato',
'plazo_de_ejec_del_contrato',
'rango_de_ejec_del_contrato',
'tiempo_adiciones_en_dias',
'tiempo_adiciones_en_meses',
'fecha_fin_ejec_contrato',
'fecha_liquidacion'
'cuantia_contrato',
'valor_total_de_adiciones',
'valor_contrato_con_adiciones',
'moneda']

In [37]:
# Columnas disponibles en el dataset
contratos_SII_df.columns
len(contratos_SII_df.columns)
# Todas las columnas, para copiar en el query
"""nombre_entidad, nit_entidad, departamento, ciudad,localizaci_n, orden, sector, rama, 
      entidad_centralizada,proceso_de_compra, id_contrato, referencia_del_contrato, estado_contrato, 
      codigo_de_categoria_principal, descripcion_del_proceso, tipo_de_contrato, modalidad_de_contratacion, 
      justificacion_modalidad_de, fecha_de_firma, fecha_de_inicio_del_contrato, fecha_de_fin_del_contrato, 
      fecha_de_inicio_de_ejecucion, fecha_de_fin_de_ejecucion, condiciones_de_entrega, tipodocproveedor, 
      documento_proveedor, proveedor_adjudicado, es_grupo, es_pyme, habilita_pago_adelantado, liquidaci_n, 
      obligaci_n_ambiental, obligaciones_postconsumo, reversion, valor_del_contrato, valor_de_pago_adelantado, 
      valor_facturado, valor_pendiente_de_pago, valor_pagado, valor_amortizado, valor_pendiente_de, 
      valor_pendiente_de_ejecucion, estado_bpin, c_digo_bpin, anno_bpin, saldo_cdp, saldo_vigencia, 
      espostconflicto, urlproceso, destino_gasto, origen_de_los_recursos, dias_adicionados, puntos_del_acuerdo, 
      pilares_del_acuerdo, nombre_representante_legal, nacionalidad_representante_legal, 
      tipo_de_identificaci_n_representante_legal, identificaci_n_representante_legal, g_nero_representante_legal, 
      presupuesto_general_de_la_nacion_pgn, sistema_general_de_participaciones, sistema_general_de_regal_as, 
      recursos_propios_alcald_as_gobernaciones_y_resguardos_ind_genas_, recursos_de_credito, recursos_propios, 
      ultima_actualizacion, codigo_entidad, fecha_inicio_liquidacion, fecha_fin_liquidacion, codigo_proveedor"""

'nombre_entidad, nit_entidad, departamento, ciudad,localizaci_n, orden, sector, rama, \n      entidad_centralizada,proceso_de_compra, id_contrato, referencia_del_contrato, estado_contrato, \n      codigo_de_categoria_principal, descripcion_del_proceso, tipo_de_contrato, modalidad_de_contratacion, \n      justificacion_modalidad_de, fecha_de_firma, fecha_de_inicio_del_contrato, fecha_de_fin_del_contrato, \n      fecha_de_inicio_de_ejecucion, fecha_de_fin_de_ejecucion, condiciones_de_entrega, tipodocproveedor, \n      documento_proveedor, proveedor_adjudicado, es_grupo, es_pyme, habilita_pago_adelantado, liquidaci_n, \n      obligaci_n_ambiental, obligaciones_postconsumo, reversion, valor_del_contrato, valor_de_pago_adelantado, \n      valor_facturado, valor_pendiente_de_pago, valor_pagado, valor_amortizado, valor_pendiente_de, \n      valor_pendiente_de_ejecucion, estado_bpin, c_digo_bpin, anno_bpin, saldo_cdp, saldo_vigencia, \n      espostconflicto, urlproceso, destino_gasto, origen_d

In [38]:
columnas_SII = ['orden',
'entidad_centralizada',
'sector',
'rama',
'nombre_entidad',
'nit_entidad',
'codigo_entidad',
'modalidad_de_contratacion',
'estado_contrato',
'justificacion_modalidad_de',
'objeto_del_contrato',
'descripcion_del_proceso',
'tipo_de_contrato',
'ciudad',
'departamento',
'localizaci_n',
'codigo_de_categoria_principal',
'tipodocproveedor',
'documento_proveedor',
'proveedor_adjudicado',
'tipo_de_identificaci_n_representante_legal',
'identificaci_n_representante_legal',
'nombre_representante_legal',
'g_nero_representante_legal',
'nacionalidad_representante_legal',
'es_grupo', 
'es_pyme',
'fecha_de_firma',
'fecha_de_inicio_del_contrato',
'dias_adicionados',
'fecha_de_fin_del_contrato',
'fecha_fin_liquidacion',
'valor_del_contrato']

In [39]:
#Test descargar solo las columnas que son utiles y estan en ambos datasets
%%time
socrata_dataset_identifier = "f789-7hwg"

Query = """
SELECT 
    uid, numero_de_constancia, numero_de_proceso, numero_de_contrato, anno_firma_contrato,
    anno_cargue_secop, nivel_entidad,orden_entidad, nombre_entidad, nit_de_la_entidad,
    c_digo_de_la_entidad, modalidad_de_contratacion, estado_del_proceso, objeto_a_contratar,
    objeto_del_contrato_a_la, detalle_del_objeto_a_contratar, tipo_de_contrato, municipio_entidad,
    departamento_entidad, id_grupo, nombre_grupo, id_familia, nombre_familia, id_clase, nombre_clase, 
    tipo_identifi_del_contratista, identificacion_del_contratista,
    nom_razon_social_contratista,  tipo_doc_representante_legal, identific_representante_legal,
    nombre_del_represen_legal, sexo_replegal, fecha_de_firma_del_contrato, fecha_ini_ejec_contrato,
    plazo_de_ejec_del_contrato, rango_de_ejec_del_contrato, tiempo_adiciones_en_dias,
    tiempo_adiciones_en_meses, fecha_fin_ejec_contrato, fecha_liquidacion, cuantia_contrato,
    valor_total_de_adiciones, valor_contrato_con_adiciones, moneda
WHERE
    fecha_de_firma_del_contrato >= '2018-01-01'
LIMIT 
    6
"""

contratos_SI = client.get(socrata_dataset_identifier, content_type="json", query=Query)

contratos_SI_DF = pd.DataFrame(pd.DataFrame.from_dict(contratos_SI))

CPU times: user 6.7 ms, sys: 893 µs, total: 7.6 ms
Wall time: 426 ms


In [40]:
#Test descargar solo las columnas que son utiles y estan en ambos datasets
%%time
socrata_dataset_identifier = "jbjy-vk9h"

Query = """
SELECT 
    orden, entidad_centralizada, sector, rama, nombre_entidad, nit_entidad, codigo_entidad,
    modalidad_de_contratacion, estado_contrato, justificacion_modalidad_de, objeto_del_contrato,
    descripcion_del_proceso, tipo_de_contrato, ciudad, departamento, localizaci_n, codigo_de_categoria_principal,
    tipodocproveedor, documento_proveedor, proveedor_adjudicado, tipo_de_identificaci_n_representante_legal,
    identificaci_n_representante_legal, nombre_representante_legal, g_nero_representante_legal,
    nacionalidad_representante_legal, es_grupo, es_pyme, fecha_de_firma, fecha_de_inicio_del_contrato,
    dias_adicionados, fecha_de_fin_del_contrato, fecha_fin_liquidacion, valor_del_contrato
WHERE
    fecha_de_firma >= '2018-01-01'
LIMIT
    100
"""

contratos_SII = client.get(socrata_dataset_identifier, content_type="json", query=Query)

contratos_SII_df = pd.DataFrame(pd.DataFrame.from_dict(contratos_SII))

CPU times: user 11.7 ms, sys: 940 µs, total: 12.6 ms
Wall time: 387 ms


In [41]:
contratos_SI_DF.head(3)

,uid,numero_de_constancia,numero_de_proceso,numero_de_contrato,anno_firma_contrato,anno_cargue_secop,nivel_entidad,orden_entidad,nombre_entidad,nit_de_la_entidad,c_digo_de_la_entidad,modalidad_de_contratacion,estado_del_proceso,objeto_a_contratar,objeto_del_contrato_a_la,detalle_del_objeto_a_contratar,tipo_de_contrato,municipio_entidad,departamento_entidad,id_grupo,nombre_grupo,id_familia,nombre_familia,id_clase,nombre_clase,tipo_identifi_del_contratista,identificacion_del_contratista,nom_razon_social_contratista,tipo_doc_representante_legal,identific_representante_legal,nombre_del_represen_legal,sexo_replegal,fecha_de_firma_del_contrato,fecha_ini_ejec_contrato,plazo_de_ejec_del_contrato,rango_de_ejec_del_contrato,tiempo_adiciones_en_dias,tiempo_adiciones_en_meses,fecha_fin_ejec_contrato,cuantia_contrato,valor_total_de_adiciones,valor_contrato_con_adiciones,moneda,fecha_liquidacion
0,18-12-7863192-7138877,18-12-7863192,2018059,2018059,1905,2018,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,META HOSPITAL DE CASTILLA LA NUEVA ESE,900004059,250150017,Contratación Directa (Ley 1150 de 2007),Celebrado,Servicios de Salud,PRESTACION DE SERVICIOS COMO PROFESIONAL EN BA...,PRESTACION DE SERVICIOS COMO PROFESIONAL EN BA...,Prestación de Servicios,Castilla La Nueva,Meta,F,[F] Servicios,8510,Servicios integrales de salud,851016,Personas de soporte de prestación de servicios...,Cédula de Ciudadanía,60376429,EDNA DEICE HURTADO PEREZ,Cédula de Ciudadanía,60376429,EDNA DEICE HURTADO PEREZ,N,2018-01-01T00:00:00.000,2018-01-01T00:00:00.000,4,M,0,2,2018-07-01T00:00:00.000,14700000,7350000,22050000,Peso Colombiano,NaN
1,18-12-7533198-6845825,18-12-7533198,5918,5918,1905,2018,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,RISARALDA ESE SALUD PEREIRA,816005003,266001463,Contratación Directa (Ley 1150 de 2007),Celebrado,Servicios de Salud,PRESTACION DE SERVICIOS PROFESIONALES COMO MED...,PRESTACION DE SERVICIOS PROFESIONALES COMO MED...,Prestación de Servicios,Pereira,Risaralda,F,[F] Servicios,8510,Servicios integrales de salud,851016,Personas de soporte de prestación de servicios...,Cédula de Ciudadanía,1144133036,VIVIAN ROCIO CORDOBA ALMARIO,Cédula de Ciudadanía,1144133036,VIVIAN ROCIO CORDOBA ALMARIO,N,2018-01-01T00:00:00.000,2018-01-01T00:00:00.000,7,M,0,0,2018-08-01T00:00:00.000,36400000,0,36400000,Peso Colombiano,NaN
2,18-12-7493899-6810252,18-12-7493899,013,013,1905,2018,TERRITORIAL,TERRITORIAL DEPARTAMENTAL DESCENTRALIZADO,TOLIMA UNIDAD DE SALUD DE IBAGUÉ,809003590,26617044,Contratación Directa (Ley 1150 de 2007),Celebrado,"Servicios de Transporte, Almacenaje y Correo",EJECUCIÓN Y DESARROLLO BAJO SU PROPIO RIESGO Y...,EJECUCIÓN Y DESARROLLO BAJO SU PROPIO RIESGO Y...,Prestación de Servicios,Dosquebradas,Risaralda,F,[F] Servicios,7814,Servicios de transporte,781415,Servicios de organización de transportes,Nit de Persona Jurídica,900318379,SERVICIOS ESPECIALES EL SOL DE LA VARIANTE LTDA,Cédula de Ciudadanía,46370868,ALBA LUZ RINCON GARRIDO,N,2018-01-01T00:00:00.000,2018-01-01T00:00:00.000,2,M,0,0,2018-03-01T00:00:00.000,32806200,0,32806200,Peso Colombiano,NaN


In [42]:
contratos_SII_df.head(3)

,orden,entidad_centralizada,sector,rama,nombre_entidad,nit_entidad,codigo_entidad,modalidad_de_contratacion,estado_contrato,justificacion_modalidad_de,objeto_del_contrato,descripcion_del_proceso,tipo_de_contrato,ciudad,departamento,localizaci_n,codigo_de_categoria_principal,tipodocproveedor,documento_proveedor,proveedor_adjudicado,tipo_de_identificaci_n_representante_legal,identificaci_n_representante_legal,nombre_representante_legal,g_nero_representante_legal,nacionalidad_representante_legal,es_grupo,es_pyme,fecha_de_firma,fecha_de_inicio_del_contrato,dias_adicionados,fecha_de_fin_del_contrato,valor_del_contrato,fecha_fin_liquidacion
0,Nacional,Centralizada,No aplica/No pertenece,Corporación Autónoma,JURISDICCION ESPECIAL DE PAZ,901140004,703840058,Contratación directa,Activo,ServiciosProfesionales,PRESTACIÓN DE SERVICIOS PROFESIONALES ESPECIAL...,PRESTACIÓN DE SERVICIOS PROFESIONALES ESPECIA...,Prestación de servicios,Bogotá,Distrito Capital de Bogotá,"Colombia, Bogotá, Bogotá",V1.93141500,NIT,79261868,Jose Arles Nova Villanueva,No Definido,79261868,José Arles Nova Villanueva,No Definido,Colombiano,No,No,2019-02-18T08:02:03.000,2019-02-01T00:00:00.000,0,2019-12-31T00:00:00.000,114393961,NaN
1,Territorial,Centralizada,Inclusión Social y Reconciliación,Ejecutivo,Secretaria Distrital de Integración Social - O...,8999990611,702271321,Contratación directa,terminado,ServiciosProfesionales,PRESTAR SERVICIOS PARA LA ATENCIÓN INTEGRAL A ...,PRESTAR SERVICIOS PARA LA ATENCIÓN INTEGRAL A ...,Prestación de servicios,No Definido,Distrito Capital de Bogotá,"Colombia, Bogotá, Bogotá",V1.80111500,Cédula de Ciudadanía,52548024,JIMENA VERGARA GARNICA,No Definido,52548024,JIMENA VERGARA GARNICA,No Definido,COLOMBIANA,No,No,2020-05-18T11:05:14.000,2020-06-01T00:00:00.000,0,2020-10-30T00:00:00.000,16390000,NaN
2,Nacional,Centralizada,Minas y Energía,Ejecutivo,SERVICIO GEOLOGICO COLOMBIANO,899999294,700765068,Contratación directa,Activo,ServiciosProfesionales,El Contratista se compromete para con el SERVI...,El Contratista se compromete para con el Servi...,Prestación de servicios,Bogotá,Distrito Capital de Bogotá,"Colombia, Bogotá, Bogotá",V1.81151901,Cédula de Ciudadanía,12749676,Dario Fernando Arcos Guerrero,No Definido,12749676,Dario Fernando Arcos Guerrero,No Definido,Colombiano,No,No,2018-12-19T21:12:01.000,2018-12-19T00:00:00.000,0,2019-08-31T00:00:00.000,59259607,NaN


In [43]:
# # Descargar todos los contratos relacionados con Servicios de Seguridad
%%time
contratos_SI_Servicios_df = pd.DataFrame()

for unspsc in UNSPSC_servicios_codes['UNSPSC'].str[3:-2].drop_duplicates().values:
    Query = Query1 + unspsc[3:] + Query2
    try:
        Aux_json = client.get(socrata_dataset_identifier, content_type="json", query=Query)
        Aux_df = pd.DataFrame(pd.DataFrame.from_dict(Aux_json))
        contratos_SI_Servicios_df = pd.concat([contratos_SI_Servicios_df, Aux_df])
        print('Added: ', unspsc, Aux_df.shape)
    except:
        print("Fail:  ", unspsc)

contratos_SI_Servicios_df.shape

NameError: ignored

In [44]:
# Descargar todos los contratos relacionados con proveedores de tecnologia, Equipos, comercializadores, importadores, etc.
%%time
contratos_SI_Equipos_df = pd.DataFrame()

for unspsc in UNSPSC_equipos_codes:
    Query = Query1 + unspsc[3:] + Query2
    try:
        Aux_json = client.get(socrata_dataset_identifier, content_type="json", query=Query)
        Aux_df = pd.DataFrame(pd.DataFrame.from_dict(Aux_json))
        contratos_SI_Equipos_df = pd.concat([contratos_SI_Equipos_df, Aux_df])
        print(unspsc, Aux_df.shape)
    except:
        print("Fail")

contratos_SI_Equipos_df.shape

NameError: ignored

In [45]:
# Descargar todos los contratos relacionados con Blindados
%%time
contratos_SI_Blindados_df = pd.DataFrame()

for unspsc in UNSPSC_blindados_codes:
    Query = Query1 + unspsc[3:] + Query2
    try:
        Aux_json = client.get(socrata_dataset_identifier, content_type="json", query=Query)
        Aux_df = pd.DataFrame(pd.DataFrame.from_dict(Aux_json))
        contratos_SI_Blindados_df = pd.concat([contratos_SI_Blindados_df, Aux_df])
        print(unspsc, Aux_df.shape)
    except:
        print("Fail")

contratos_SI_Blindados_df.shape

NameError: ignored

In [46]:
# Descargar todos los contratos relacionados con Armas
%%time
contratos_SI_Armas_df = pd.DataFrame()

for unspsc in UNSPSC_armas_codes:
    Query = Query1 + unspsc[3:] + Query2
    try:
        Aux_json = client.get(socrata_dataset_identifier, content_type="json", query=Query)
        Aux_df = pd.DataFrame(pd.DataFrame.from_dict(Aux_json))
        contratos_SI_Armas_df = pd.concat([contratos_SI_Armas_df, Aux_df])
        print(unspsc, Aux_df.shape)
    except:
        print("Fail")

contratos_SI_Armas_df.shape

NameError: ignored

In [47]:
# Descargar todos los contratos relacionados con Servicios de Seguridad
%%time
contratos_SII_Servicios_df = pd.DataFrame()

for unspsc in UNSPSC_servicios_codes:
    Query = Query1 + unspsc + Query2
    try:
        Aux_json = client.get(socrata_dataset_identifier, content_type="json", query=Query)
        Aux_df = pd.DataFrame(pd.DataFrame.from_dict(Aux_json))
        contratos_SII_Servicios_df = pd.concat([contratos_SII_Servicios_df, Aux_df])
        print(unspsc, Aux_df.shape)
    except:
        print("Fail")

contratos_SII_Servicios_df.shape

NameError: ignored

In [48]:
# Descargar todos los contratos relacionados con proveedores de tecnologia, Equipos, comercializadores, importadores, etc.
%%time
contratos_SII_Equipos_df = pd.DataFrame()

for unspsc in UNSPSC_equipos_codes:
    Query = Query1 + unspsc + Query2
    try:
        Aux_json = client.get(socrata_dataset_identifier, content_type="json", query=Query)
        Aux_df = pd.DataFrame(pd.DataFrame.from_dict(Aux_json))
        contratos_SII_Equipos_df = pd.concat([contratos_SII_Equipos_df, Aux_df])
        print(unspsc, Aux_df.shape)
    except:
        print("Fail")

contratos_SII_Equipos_df.shape

NameError: ignored

In [49]:
# Descargar todos los contratos relacionados con Blindados
%%time
contratos_SII_Blindados_df = pd.DataFrame()

for unspsc in UNSPSC_blindados_codes:
    Query = Query1 + unspsc + Query2
    try:
        Aux_json = client.get(socrata_dataset_identifier, content_type="json", query=Query)
        Aux_df = pd.DataFrame(pd.DataFrame.from_dict(Aux_json))
        contratos_SII_Blindados_df = pd.concat([contratos_SII_Blindados_df, Aux_df])
        print(unspsc, Aux_df.shape)
    except:
        print("Fail")

contratos_SII_Blindados_df.shape

NameError: ignored

In [50]:
# Descargar todos los contratos relacionados con Armas
%%time
contratos_SII_Armas_df = pd.DataFrame()

for unspsc in UNSPSC_armas_codes:
    Query = Query1 + unspsc + Query2
    try:
        Aux_json = client.get(socrata_dataset_identifier, content_type="json", query=Query)
        Aux_df = pd.DataFrame(pd.DataFrame.from_dict(Aux_json))
        contratos_SII_Armas_df = pd.concat([contratos_SII_Armas_df, Aux_df])
        print(unspsc, Aux_df.shape)
    except:
        print("Fail")

contratos_SII_Armas_df.shape

NameError: ignored

## Descargar SECOP_I

In [ ]:
socrata_dataset_identifier = "f789-7hwg"

Query1 = """
SELECT 
    uid, numero_de_constancia, numero_de_proceso, numero_de_contrato, anno_firma_contrato,
    anno_cargue_secop, nivel_entidad,orden_entidad, nombre_entidad, nit_de_la_entidad,
    c_digo_de_la_entidad, modalidad_de_contratacion, estado_del_proceso, objeto_a_contratar,
    objeto_del_contrato_a_la, detalle_del_objeto_a_contratar, tipo_de_contrato, municipio_entidad,
    departamento_entidad, id_grupo, nombre_grupo, id_familia, nombre_familia, id_clase, nombre_clase, 
    tipo_identifi_del_contratista, identificacion_del_contratista,
    nom_razon_social_contratista,  tipo_doc_representante_legal, identific_representante_legal,
    nombre_del_represen_legal, sexo_replegal, fecha_de_firma_del_contrato, fecha_ini_ejec_contrato,
    plazo_de_ejec_del_contrato, rango_de_ejec_del_contrato, tiempo_adiciones_en_dias,
    tiempo_adiciones_en_meses, fecha_fin_ejec_contrato, fecha_liquidacion, cuantia_contrato,
    valor_total_de_adiciones, valor_contrato_con_adiciones, moneda
WHERE 
    fecha_de_firma_del_contrato >= '2000-01-01'
AND 
    id_clase LIKE '"""

Query2 = """%' LIMIT 25000"""

In [52]:
# Descargar todos los contratosde SECOP-I que nos interesan:
%%time
contratos_SI_df = pd.DataFrame()

for tipo_servicio, lista in UNSPSC_codes_dict.items():
    print (tipo_servicio)
    for unspsc in lista:
        Query = Query1 + unspsc[3:] + Query2
        try:
            Aux_json = client.get(socrata_dataset_identifier, content_type="json", query=Query)
            Aux_df = pd.DataFrame(pd.DataFrame.from_dict(Aux_json))
            Aux_df['Tipo_servicio'] = tipo_servicio
            contratos_SI_df = pd.concat([contratos_SI_df, Aux_df])
            print('Added: ', unspsc[3:], Aux_df.shape, '    total: ', contratos_SI_df.shape)
        except:
            print("Fail:  ", unspsc[3:])


Servicio de Vigilancia y Seguridad Privada
Fail:   92101501
Added:  9212 (22492, 45)     total:  (22492, 45)
Added:  921215 (16634, 45)     total:  (39126, 45)
Added:  921217 (5224, 45)     total:  (44350, 45)
Fail:   93121503
Servicio de Asesoria, Consultoria e Investigacion
Fail:   80111702
Fail:   81141801
Fail:   921216
Servicio de Capacitacion y Entrenamiento en Vigilancia y Seguridad Privada
Fail:   86101709
Fabricacion, Importacion, Comercio, Arriendo e Instalación de Equipos para la Vigilancia y la Seguridad Privada
Fail:   461516
Fail:   461518
Added:  4617 (4179, 45)     total:  (48529, 45)
Fail:   461715
Added:  461716 (3484, 45)     total:  (52013, 45)
Fail:   721517
Servicio de Transporte de Valores
Fail:   78101503
Fail:   78101705
Fail:   921218
Servicio de Fabricación y/o Arrendamiento de Vehículos Blindados
Fail:   25101920
Suministro de Armas
Fail:   460000
Fail:   4610
Fail:   461015
Fail:   461016
Fail:   461017
Fail:   461018
CPU times: user 4.26 s, sys: 564 ms, to

In [53]:
Query2 = """%' LIMIT 10000"""

In [54]:
%%time
for tipo_servicio, lista in UNSPSC_codes_dict.items():
    print (tipo_servicio)
    for unspsc in lista:
        Query = Query1 + unspsc[3:] + Query2
        try:
            Aux_json = client.get(socrata_dataset_identifier, content_type="json", query=Query)
            Aux_df = pd.DataFrame(pd.DataFrame.from_dict(Aux_json))
            Aux_df['Tipo_servicio'] = tipo_servicio
            contratos_SI_df = pd.concat([contratos_SI_df, Aux_df])  #ignore_index=True
            print('Added: ', unspsc[3:], Aux_df.shape, '    total: ', contratos_SI_df.shape)
        except:
            print("Fail:  ", unspsc[3:])

Servicio de Vigilancia y Seguridad Privada
Fail:   92101501
Added:  9212 (10000, 45)     total:  (62013, 45)
Added:  921215 (10000, 45)     total:  (72013, 45)
Added:  921217 (5224, 45)     total:  (77237, 45)
Fail:   93121503
Servicio de Asesoria, Consultoria e Investigacion
Fail:   80111702
Fail:   81141801
Fail:   921216
Servicio de Capacitacion y Entrenamiento en Vigilancia y Seguridad Privada
Fail:   86101709
Fabricacion, Importacion, Comercio, Arriendo e Instalación de Equipos para la Vigilancia y la Seguridad Privada
Fail:   461516
Fail:   461518
Added:  4617 (4179, 45)     total:  (81416, 45)
Fail:   461715
Added:  461716 (3484, 45)     total:  (84900, 45)
Fail:   721517
Servicio de Transporte de Valores
Fail:   78101503
Fail:   78101705
Fail:   921218
Servicio de Fabricación y/o Arrendamiento de Vehículos Blindados
Fail:   25101920
Suministro de Armas
Fail:   460000
Fail:   4610
Fail:   461015
Fail:   461016
Fail:   461017
Fail:   461018
CPU times: user 3.66 s, sys: 298 ms, to

In [55]:
Query2 = """%' LIMIT 1000"""

In [56]:
%%time
for tipo_servicio, lista in UNSPSC_codes_dict.items():
    print (tipo_servicio)
    for unspsc in lista:
        Query = Query1 + unspsc[3:] + Query2
        try:
            Aux_json = client.get(socrata_dataset_identifier, content_type="json", query=Query)
            Aux_df = pd.DataFrame(pd.DataFrame.from_dict(Aux_json))
            Aux_df['Tipo_servicio'] = tipo_servicio
            contratos_SI_df = pd.concat([contratos_SI_df, Aux_df])
            print('Added: ', unspsc[3:], Aux_df.shape, '    total: ', contratos_SI_df.shape)
        except:
            print("Fail:  ", unspsc[3:])

Servicio de Vigilancia y Seguridad Privada
Added:  92101501 (0, 1)     total:  (84900, 45)
Added:  9212 (1000, 45)     total:  (85900, 45)
Added:  921215 (1000, 45)     total:  (86900, 45)
Added:  921217 (1000, 45)     total:  (87900, 45)
Added:  93121503 (0, 1)     total:  (87900, 45)
Servicio de Asesoria, Consultoria e Investigacion
Added:  80111702 (0, 1)     total:  (87900, 45)
Added:  81141801 (0, 1)     total:  (87900, 45)
Fail:   921216
Servicio de Capacitacion y Entrenamiento en Vigilancia y Seguridad Privada
Added:  86101709 (0, 1)     total:  (87900, 45)
Fabricacion, Importacion, Comercio, Arriendo e Instalación de Equipos para la Vigilancia y la Seguridad Privada
Fail:   461516
Added:  461518 (29, 45)     total:  (87929, 45)
Added:  4617 (1000, 45)     total:  (88929, 45)
Added:  461715 (695, 45)     total:  (89624, 45)
Added:  461716 (1000, 45)     total:  (90624, 45)
Added:  721517 (770, 45)     total:  (91394, 45)
Servicio de Transporte de Valores
Added:  78101503 (0, 1) 

In [57]:
contratos_SI_df = contratos_SI_df.drop_duplicates(ignore_index=True)
contratos_SI_df.shape

(27786, 45)

In [58]:
contratos_SI_df = contratos_SI_df.sort_values(by = 'cuantia_contrato', ascending = False)
contratos_SI_df = contratos_SI_df.drop_duplicates(subset = ['uid', 'numero_de_constancia', 'numero_de_proceso','numero_de_contrato',
                                          'anno_firma_contrato', 'anno_cargue_secop', 'nit_de_la_entidad', 'identificacion_del_contratista', 
                                          'identific_representante_legal', 'fecha_de_firma_del_contrato'], keep='first', ignore_index=True)
contratos_SI_df.shape

(27646, 45)

## Descargar SECOP_II

In [59]:
socrata_dataset_identifier = "jbjy-vk9h"

Query3 = """
SELECT 
    proceso_de_compra, id_contrato, referencia_del_contrato, anno_bpin, orden, 
    entidad_centralizada, sector, rama, nombre_entidad, nit_entidad, codigo_entidad,
    modalidad_de_contratacion, estado_contrato, justificacion_modalidad_de, objeto_del_contrato,
    descripcion_del_proceso, tipo_de_contrato, ciudad, departamento, localizaci_n, codigo_de_categoria_principal,
    tipodocproveedor, documento_proveedor, proveedor_adjudicado, tipo_de_identificaci_n_representante_legal,
    identificaci_n_representante_legal, nombre_representante_legal, g_nero_representante_legal,
    nacionalidad_representante_legal, es_grupo, es_pyme, fecha_de_firma, fecha_de_inicio_del_contrato, fecha_de_inicio_de_ejecucion, 
    fecha_de_fin_de_ejecucion, dias_adicionados, fecha_de_fin_del_contrato, fecha_fin_liquidacion, valor_del_contrato
WHERE 
    fecha_de_firma >= '2000-01-01' 
AND 
    codigo_de_categoria_principal LIKE '"""

Query4 = """%' LIMIT 10000"""

In [60]:
# Descargar todos los contratosde SECOP-II que nos interesan:
%%time
contratos_SII_df = pd.DataFrame()

for tipo_servicio, lista in UNSPSC_codes_dict.items():
    print (tipo_servicio)
    for unspsc in lista:
        Query = Query3 + unspsc + Query4
        try:
            Aux_json = client.get(socrata_dataset_identifier, content_type="json", query=Query)
            Aux_df = pd.DataFrame(pd.DataFrame.from_dict(Aux_json))
            Aux_df['Tipo_servicio'] = tipo_servicio
            contratos_SII_df = pd.concat([contratos_SII_df, Aux_df])   #ignore_index=True
            print('Added: ', unspsc, Aux_df.shape, '    total: ', contratos_SII_df.shape)
        except:
            print("Fail:  ", unspsc)

Servicio de Vigilancia y Seguridad Privada
Added:  V1.92101501 (455, 40)     total:  (455, 40)
Added:  V1.9212 (2159, 40)     total:  (2614, 40)
Added:  V1.921215 (1610, 40)     total:  (4224, 40)
Added:  V1.921217 (393, 40)     total:  (4617, 40)
Added:  V1.93121503 (0, 1)     total:  (4617, 40)
Servicio de Asesoria, Consultoria e Investigacion
Added:  V1.80111702 (40, 40)     total:  (4657, 40)
Added:  V1.81141801 (51, 40)     total:  (4708, 40)
Added:  V1.921216 (25, 40)     total:  (4733, 40)
Servicio de Capacitacion y Entrenamiento en Vigilancia y Seguridad Privada
Added:  V1.86101709 (125, 40)     total:  (4858, 40)
Fabricacion, Importacion, Comercio, Arriendo e Instalación de Equipos para la Vigilancia y la Seguridad Privada
Added:  V1.461516 (59, 40)     total:  (4917, 40)
Added:  V1.461518 (7, 40)     total:  (4924, 40)
Added:  V1.4617 (806, 40)     total:  (5730, 40)
Added:  V1.461715 (121, 40)     total:  (5851, 40)
Added:  V1.461716 (685, 40)     total:  (6536, 40)
Added:  

In [61]:
contratos_SII_df = contratos_SII_df.drop_duplicates(ignore_index=True)
contratos_SII_df.shape

(4126, 40)

In [62]:
contratos_SII_df.drop_duplicates(subset = ['proceso_de_compra', 'referencia_del_contrato', 'nit_entidad', 'documento_proveedor', 'identificaci_n_representante_legal', 'fecha_de_firma']).shape

(3970, 40)

In [63]:
contratos_SII_df = contratos_SII_df.sort_values(by = 'valor_del_contrato', ascending = False)
contratos_SII_df = contratos_SII_df.drop_duplicates(subset = ['proceso_de_compra', 'referencia_del_contrato', 'nit_entidad', 'documento_proveedor', 'identificaci_n_representante_legal', 'fecha_de_firma'], keep='first', ignore_index=True)
contratos_SII_df.shape

(3970, 40)

In [64]:
contratos_SI_df['nombre_clase'].unique()

array(['Equipo de vigilancia y detección',
       'Cerraduras elementos de seguridad y accesorios',
       'Servicios de instalación de sistemas de seguridad física e industrial',
       'Servicios de sistemas de seguridad', 'Servicios de guardias',
       'Servicios de seguridad marina',
       'Sistemas de manipulación de munición',
       'Servicios de Carros blindados y transporte de valores',
       'Armas de fuego', 'Servicios de detectives', 'Munición',
       'Accesorios de armas y municiones',
       'Equipo y accesorios y suministros de control de explosivos'],
      dtype=object)

# Step 2: Consolidar SECOP_I y SECOP_II

In [65]:
SI_df = contratos_SI_df.copy()
SII_df = contratos_SII_df.copy()

### Eliminar Contratos que no sirven para este caso

In [66]:
#SII_df.reset_index(drop=True)

In [67]:
SI_df.drop(SI_df[SI_df['id_clase'].str.contains('721516')].index, inplace = True)
SI_df[SI_df['id_clase'].str.contains('721516')]['detalle_del_objeto_a_contratar']

Series([], Name: detalle_del_objeto_a_contratar, dtype: object)

In [68]:
SII_df.drop(SII_df[SII_df['codigo_de_categoria_principal'].str.contains('V1.721516')].index, inplace = True)
SII_df[SII_df['codigo_de_categoria_principal'].str.contains('V1.721516')]

,proceso_de_compra,id_contrato,referencia_del_contrato,anno_bpin,orden,entidad_centralizada,sector,rama,nombre_entidad,nit_entidad,codigo_entidad,modalidad_de_contratacion,estado_contrato,justificacion_modalidad_de,objeto_del_contrato,descripcion_del_proceso,tipo_de_contrato,ciudad,departamento,localizaci_n,codigo_de_categoria_principal,tipodocproveedor,documento_proveedor,proveedor_adjudicado,tipo_de_identificaci_n_representante_legal,identificaci_n_representante_legal,nombre_representante_legal,g_nero_representante_legal,nacionalidad_representante_legal,es_grupo,es_pyme,fecha_de_firma,fecha_de_inicio_del_contrato,fecha_de_inicio_de_ejecucion,fecha_de_fin_de_ejecucion,dias_adicionados,fecha_de_fin_del_contrato,fecha_fin_liquidacion,valor_del_contrato,Tipo_servicio


In [69]:
SI_df.drop(SI_df[SI_df['id_clase'].str.contains('841220')].index, inplace = True)
SI_df[SI_df['id_clase'].str.contains('841220')]['detalle_del_objeto_a_contratar']

Series([], Name: detalle_del_objeto_a_contratar, dtype: object)

In [70]:
SII_df.drop(SII_df[SII_df['codigo_de_categoria_principal'].str.contains('V1.841220')].index, inplace = True)
SII_df[SII_df['codigo_de_categoria_principal'].str.contains('V1.841220')]['descripcion_del_proceso']

Series([], Name: descripcion_del_proceso, dtype: object)

In [71]:
SII_df[SII_df['codigo_de_categoria_principal']=='V1.921215']

,proceso_de_compra,id_contrato,referencia_del_contrato,anno_bpin,orden,entidad_centralizada,sector,rama,nombre_entidad,nit_entidad,codigo_entidad,modalidad_de_contratacion,estado_contrato,justificacion_modalidad_de,objeto_del_contrato,descripcion_del_proceso,tipo_de_contrato,ciudad,departamento,localizaci_n,codigo_de_categoria_principal,tipodocproveedor,documento_proveedor,proveedor_adjudicado,tipo_de_identificaci_n_representante_legal,identificaci_n_representante_legal,nombre_representante_legal,g_nero_representante_legal,nacionalidad_representante_legal,es_grupo,es_pyme,fecha_de_firma,fecha_de_inicio_del_contrato,fecha_de_inicio_de_ejecucion,fecha_de_fin_de_ejecucion,dias_adicionados,fecha_de_fin_del_contrato,fecha_fin_liquidacion,valor_del_contrato,Tipo_servicio


In [72]:
SII_df[SII_df['sector']=='Inteligencia Estratégica y Contrainteligencia'][ 'objeto_del_contrato']

922     Aunar esfuerzos humanos técnicos logísticos y ...
2815    Aunar esfuerzos humanos técnicos logísticos y ...
Name: objeto_del_contrato, dtype: object

## Ajustar Nombre, Formato y Consistencia de variables

In [73]:
SII_df.rename(columns = {
    'anno_bpin' : 'anno_cargue_secop',
    'ciudad' : 'municipio_entidad',
    'departamento' : 'departamento_entidad',
    'tipodocproveedor' : 'tipo_doc_contratista',
    'documento_proveedor' : 'identificacion_contratista',
    'proveedor_adjudicado' : 'nom_razon_social_contratista',
    'tipo_de_identificaci_n_representante_legal' : 'tipo_doc_representante_legal',
    'identificaci_n_representante_legal' : 'identificacion_representante_legal',
    'fecha_de_firma' : 'fecha_de_firma_del_contrato',
    'dias_adicionados' : 'tiempo_adiciones_en_dias',
    'g_nero_representante_legal' : 'genero_representante_legal'
    }, inplace = True)

In [74]:
SI_df.rename(columns = {
    'nivel_entidad' : 'orden',
    'orden_entidad' : 'entidad_centralizada',
    'nit_de_la_entidad' : 'nit_entidad',
    'c_digo_de_la_entidad' : 'codigo_entidad',
    'estado_del_proceso' : 'estado_contrato',
    'objeto_a_contratar' : 'justificacion_modalidad_de',
    'objeto_del_contrato_a_la' : 'objeto_del_contrato',
    'detalle_del_objeto_a_contratar' : 'descripcion_del_proceso',
    'tipo_identifi_del_contratista' : 'tipo_doc_contratista',
    'identificacion_del_contratista' : 'identificacion_contratista',
    'identific_representante_legal' : 'identificacion_representante_legal',
    'nombre_del_represen_legal' : 'nombre_representante_legal',
    'sexo_replegal' : 'genero_representante_legal',
    'fecha_ini_ejec_contrato' : 'fecha_de_inicio_del_contrato',
    'fecha_fin_ejec_contrato' : 'fecha_de_fin_del_contrato',
    'cuantia_contrato' : 'valor_del_contrato',
    }, inplace = True)

### Variable orden entidad y otros descriptores

In [75]:
SI_df['orden'] = SI_df['orden'].replace(['NACIONAL','TERRITORIAL'],['Nacional','Territorial'])

In [76]:
SI_df['orden'].unique()

array(['Territorial', 'Nacional', 'No Definido'], dtype=object)

In [77]:
SI_df.loc[SI_df['entidad_centralizada'].str.contains('TERRITORIAL '),'entidad_centralizada'] = SI_df['entidad_centralizada'].str[12:]
SI_df.loc[SI_df['entidad_centralizada'].str.contains('NACIONAL '),'entidad_centralizada'] = SI_df['entidad_centralizada'].str[9:]

In [78]:
SI_df.loc[SI_df['modalidad_de_contratacion']=='Régimen Especial' ,'modalidad_de_contratacion'] = 'Contratación régimen especial'
SI_df.loc[SI_df['modalidad_de_contratacion']=='Licitación obra pública' ,'modalidad_de_contratacion'] = 'Licitación Obra pública'
SII_df.loc[SII_df['modalidad_de_contratacion']=='Licitación pública Obra Publica' ,'modalidad_de_contratacion'] = 'Licitación Obra pública'
SII_df.loc[SII_df['modalidad_de_contratacion']=='Licitación pública' ,'modalidad_de_contratacion'] = 'Licitación Pública'
SII_df.loc[SII_df['modalidad_de_contratacion']=='Contratación directa' ,'modalidad_de_contratacion'] = 'Contratación Directa (Ley 1150 de 2007)'
SII_df.loc[SII_df['modalidad_de_contratacion']=='Concurso de méritos abierto' ,'modalidad_de_contratacion'] = 'Concurso de Méritos Abierto'
SII_df.loc[SII_df['modalidad_de_contratacion']=='Selección Abreviada de Menor Cuantía' ,'modalidad_de_contratacion'] = 'Selección Abreviada de Menor Cuantía (Ley 1150 de 2007)'

In [79]:
SI_df.loc[SI_df['tipo_de_contrato']=='Suministro' ,'tipo_de_contrato'] = 'Suministros'
SII_df.loc[SII_df['tipo_de_contrato']=='Prestación de servicios' ,'tipo_de_contrato'] = 'Prestación de Servicios'

In [80]:
SII_df.loc[SII_df['departamento_entidad']=='Distrito Capital de Bogotá' ,'departamento_entidad'] = 'Bogotá D.C.'

In [81]:
SI_df['genero_representante_legal'] = SI_df['genero_representante_legal'].replace(['N','2', '1', '3'],['No Definido','Hombre', 'Mujer', 'Otro'])

In [82]:
SII_df['nacionalidad_representante_legal'] = SII_df['nacionalidad_representante_legal'].str.lower()
SII_df.loc[SII_df['nacionalidad_representante_legal'].str.contains('olombia'),'nacionalidad_representante_legal'] = 'colombiana'
SII_df.loc[SII_df['nacionalidad_representante_legal'].str.contains('col|pasca|bogota|cucuta|barranquilla'),'nacionalidad_representante_legal'] = 'colombiana'
SII_df.loc[SII_df['nacionalidad_representante_legal'].str.contains('estado|america'),'nacionalidad_representante_legal'] = 'estadounidense'

### Variable UNSPSC

In [83]:
#UNSPSC_codes.loc['V1.92101501', 'Descripcion']
UNSPSC_codes_dict2 = UNSPSC_codes.to_dict()
#UNSPSC_codes_dict2
UNSPSC_codes_dict2['Descripcion']['V1.92121901']

'Servicio de botes de seguridad'

In [84]:
#pd.Series('V1.'+SI_df['id_clase'].add('00')) #.add_prefix('V1.') # doesnt work,  only modifies the index
#UNSPSC_codes.loc[UNSPSC_codes['UNSPSC'] == 'V1.' + '921200' + '00', 'Descripcion'] #it doesnt work without assigning first
#series1  = SI_df['codigo_de_categoria_principal'].map(lambda x: UNSPSC_codes.loc[x]) #No funciona, devuelve todas las filas de UNSPSC_codes
#series1 = [col[3:] for col in UNSPSC_servicios_codes]  #this is a list (iterable)

SI_df['codigo_de_categoria_principal'] = 'V1.' + SI_df['id_clase'] + '00'
SI_df['codigo_de_categoria_principal'].unique()

array(['V1.46171600', 'V1.46171500', 'V1.72151700', 'V1.92121700',
       'V1.92121500', 'V1.92121900', 'V1.46101700', 'V1.92121800',
       'V1.46101500', 'V1.92121600', 'V1.46101600', 'V1.46101800',
       'V1.46151800'], dtype=object)

In [85]:
SII_df['id_clase'] = SII_df['codigo_de_categoria_principal'].str[3:-2]    #= SII_DF['codigo_de_categoria_principal'].str[-8:]
SII_df['nombre_clase'] = SII_df['codigo_de_categoria_principal'].map(lambda x: UNSPSC_codes_dict2['Descripcion'][x])     
SII_df['id_clase'].unique()

array(['921217', '461716', '461018', '921015', '721517', '921215',
       '461715', '461015', '861017', '921218', '461016', '921216',
       '461516', '811418', '801117', '251019', '461518', '461017'],
      dtype=object)

In [86]:
SI_df['id_clase'].unique()

array(['461716', '461715', '721517', '921217', '921215', '921219',
       '461017', '921218', '461015', '921216', '461016', '461018',
       '461518'], dtype=object)

In [87]:
SII_df['id_clase'].unique()

array(['921217', '461716', '461018', '921015', '721517', '921215',
       '461715', '461015', '861017', '921218', '461016', '921216',
       '461516', '811418', '801117', '251019', '461518', '461017'],
      dtype=object)

In [88]:
SII_df['codigo_de_categoria_principal'].unique()

array(['V1.92121704', 'V1.46171625', 'V1.46101800', 'V1.46171610',
       'V1.46171604', 'V1.92101501', 'V1.46171619', 'V1.92121700',
       'V1.72151704', 'V1.92121504', 'V1.92121500', 'V1.46171511',
       'V1.46171602', 'V1.72151700', 'V1.46101505', 'V1.46171501',
       'V1.46171600', 'V1.86101709', 'V1.92121701', 'V1.46171622',
       'V1.46171621', 'V1.46171506', 'V1.92121800', 'V1.72151702',
       'V1.72151701', 'V1.46171509', 'V1.46101600', 'V1.92121502',
       'V1.92121604', 'V1.92121801', 'V1.92121600', 'V1.46151600',
       'V1.46171633', 'V1.46101500', 'V1.81141801', 'V1.46101503',
       'V1.46171624', 'V1.46151608', 'V1.46101501', 'V1.92121702',
       'V1.92121601', 'V1.46171507', 'V1.80111702', 'V1.46101601',
       'V1.46171514', 'V1.46171612', 'V1.72151703', 'V1.46171637',
       'V1.25101920', 'V1.46151800', 'V1.46171606', 'V1.46151604',
       'V1.46171500', 'V1.92121503', 'V1.46101802', 'V1.46171636',
       'V1.46171616', 'V1.46101801', 'V1.46171626', 'V1.461716

### Variable mes y año

In [89]:
SI_df['fecha_de_firma_del_contrato'].str[:4].unique()

array(['2019', '2021', '2018', '2022', '2017', '2015', '2016', '2020',
       '2014', '2005', '2009', '2013', '2012', '2008', '2010', '2003',
       '2002'], dtype=object)

In [90]:
SI_df['anno_firma_contrato'].unique()

array(['1905'], dtype=object)

In [91]:
SI_df['anno_firma_contrato'] = SI_df['fecha_de_firma_del_contrato'].str[:4]
SI_df['anno_firma_contrato'].unique()

array(['2019', '2021', '2018', '2022', '2017', '2015', '2016', '2020',
       '2014', '2005', '2009', '2013', '2012', '2008', '2010', '2003',
       '2002'], dtype=object)

In [92]:
SII_df['fecha_de_firma_del_contrato'].str[:4].unique()

array(['2021', '2022', '2019', '2018', '2020', '2017', '2016'],
      dtype=object)

In [93]:
SII_df['fecha_de_firma_del_contrato'].str[:4].value_counts()

2021    1079
2022     962
2020     732
2018     551
2019     525
2017     113
2016       8
Name: fecha_de_firma_del_contrato, dtype: int64

In [94]:
SII_df['anno_firma_contrato'] = SII_df['fecha_de_firma_del_contrato'].str[:4]
SII_df['anno_firma_contrato'].unique()

array(['2021', '2022', '2019', '2018', '2020', '2017', '2016'],
      dtype=object)

In [95]:
# Investigar la duración de los contratos
#series2 =  pd.to_datetime(SII_df['fecha_de_fin_de_ejecucion']).dt.day - pd.to_datetime(SII_df['fecha_de_inicio_de_ejecucion']).dt.day
#series2.value_counts()
#series1 =  pd.to_datetime(SII_df['fecha_de_fin_del_contrato']).dt.day - pd.to_datetime(SII_df['fecha_de_inicio_del_contrato']).dt.day
#series1.value_counts()
#SII_df['dias_adicionados']
#SII_df['dias_adicionados'].astype + pd.to_datetime(SII_df['fecha_de_fin_del_contrato']).dt.day - pd.to_datetime(SII_df['fecha_de_inicio_del_contrato']).dt.day
#SII_df['tiempo_adiciones_en_meses'] = abs(SII_df['dias_adicionados'].astype(int)/30)

In [96]:
# Asignar el plazo de ejecución de contratos para los contratos que no lo tienen:

SII_df['fecha_de_fin_del_contrato'] = pd.to_datetime(SII_df['fecha_de_fin_del_contrato'])
SII_df['fecha_de_inicio_del_contrato'] = pd.to_datetime(SII_df['fecha_de_inicio_del_contrato'])
SII_df['plazo_de_ejec_del_contrato'] = pd.to_numeric(1 + (SII_df['fecha_de_fin_del_contrato'] - SII_df['fecha_de_inicio_del_contrato']).dt.days)
SII_df['plazo_de_ejec_del_contrato'].describe()

count   3,859.00
mean      229.17
std       224.56
min         1.00
25%        64.00
50%       192.00
75%       324.50
max     1,827.00
Name: plazo_de_ejec_del_contrato, dtype: float64

In [97]:
SII_df.loc[SII_df['plazo_de_ejec_del_contrato'].isna(), 'plazo_de_ejec_del_contrato'] = 1 + (pd.to_datetime(SII_df['fecha_de_fin_del_contrato']) - pd.to_datetime(SII_df['fecha_de_firma_del_contrato'].str[:10])).dt.days


In [98]:
SII_df['rango_de_ejec_del_contrato'] = 'D'

In [99]:
# arreglar valores extraños de los plazos de ejecución. Se asignó el máximo plazo como 72 meses.

#SI_df['plazo_de_ejec_del_contrato']
#SI_df['rango_de_ejec_del_contrato'].unique()
#SI_df[(SI_df['rango_de_ejec_del_contrato']=='M')]['plazo_de_ejec_del_contrato'].value_counts()

SI_df.loc[(SI_df['rango_de_ejec_del_contrato']=='M') & (SI_df['plazo_de_ejec_del_contrato'].astype(int)>72), 'rango_de_ejec_del_contrato'] = 'D'

In [100]:
# Arreglar la consistencia de la variable "Plazo de Ejecución" para que toda esté en dias

SI_df.loc[SI_df['rango_de_ejec_del_contrato']== 'M', 'plazo_de_ejec_del_contrato'] = SI_df['plazo_de_ejec_del_contrato'].astype(int)*30
SI_df.loc[SI_df['rango_de_ejec_del_contrato']== 'D', 'plazo_de_ejec_del_contrato'] = pd.to_numeric(SI_df['plazo_de_ejec_del_contrato'])
SI_df.loc[SI_df['rango_de_ejec_del_contrato']== 'M', 'rango_de_ejec_del_contrato'] = 'D'
SI_df['plazo_de_ejec_del_contrato'].unique()

array([90, 2, 30, 5, 60, 15, 20, 128, 180, 300, 3, 1, 10, 31, 270, 13,
       353, 150, 8, 711, 151, 21, 360, 140, 210, 109, 164, 33, 345, 240,
       22, 276, 25, 307, 103, 330, 4, 116, 236, 78, 130, 44, 225, 6, 324,
       69, 72, 450, 89, 0, 120, 115, 165, 3350, 12, 40, 339, 365, 42,
       1230, 302, 229, 26, 265, 43, 84, 729, 237, 19, 228, 315, 264, 38,
       319, 316, 245, 14, 285, 263, 75, 425, 47, 35, 129, 295, 45, 18,
       720, 176, 366, 27, 16, 80, 540, 321, 368, 351, 23, 86, 327, 174,
       252, 340, 1440, 173, 178, 274, 76, 85, 201, 348, 34, 202, 7, 219,
       77, 117, 298, 39, 333, 172, 136, 190, 28, 267, 58, 900, 292, 175,
       250, 24, 216, 36, 338, 97, 145, 329, 317, 87, 323, 390, 48, 100,
       205, 11, 166, 290, 358, 474, 204, 170, 54, 92, 126, 169, 183, 179,
       221, 161, 29, 127, 334, 288, 9, 233, 50, 195, 17, 304, 114, 1800,
       70, 62, 359, 275, 99, 163, 162, 105, 66, 301, 110, 279, 220, 155,
       570, 152, 299, 108, 171, 344, 364, 185, 51, 253, 34

### Variable Departamento

In [ ]:
contratos_SECOP_df.loc[contratos_SECOP_df['departamento_entidad']=='Bogotá DC', 'departamento_entidad'] = 'Bogotá D.C.'
contratos_SECOP_df.loc[contratos_SECOP_df['departamento_entidad']=='Norte de Santander', 'departamento_entidad'] =  'Norte De Santander'
contratos_SECOP_df.loc[contratos_SECOP_df['departamento_entidad']=='San Andrés, Providencia y Santa Catalina', 'departamento_entidad'] = 'San Andrés Providencia y Santa Catalina'
contratos_SECOP_df.loc[contratos_SECOP_df['departamento_entidad']=='Colombia', 'departamento_entidad'] = 'No Definido'
len(contratos_SECOP_df['departamento_entidad'].unique())

## Concatenar dataframes

In [101]:
[ col for col in SI_df.columns if col not in SII_df.columns ]

['uid',
 'numero_de_constancia',
 'numero_de_proceso',
 'numero_de_contrato',
 'id_grupo',
 'nombre_grupo',
 'id_familia',
 'nombre_familia',
 'tiempo_adiciones_en_meses',
 'valor_total_de_adiciones',
 'valor_contrato_con_adiciones',
 'moneda',
 'fecha_liquidacion']

In [102]:
[ col for col in SII_df.columns if col not in SI_df.columns ]

['proceso_de_compra',
 'id_contrato',
 'referencia_del_contrato',
 'sector',
 'rama',
 'localizaci_n',
 'nacionalidad_representante_legal',
 'es_grupo',
 'es_pyme',
 'fecha_de_inicio_de_ejecucion',
 'fecha_de_fin_de_ejecucion',
 'fecha_fin_liquidacion']

In [103]:
SI_df['Database']="SECOP_I"
SII_df['Database']="SECOP_II"

In [160]:
cols = [ col for col in SI_df.columns if col in SII_df.columns ]
SECOP_df = pd.concat([SI_df, SII_df], verify_integrity=True, ignore_index=1)
SECOP_df.shape

(31616, 59)

In [ ]:
Armas_df = SECOP_df[SECOP_df['Tipo_servicio']=='Suministro de Armas']
Armas_df.shape

In [166]:
Armas_df.head(20)

,uid,numero_de_constancia,numero_de_proceso,numero_de_contrato,anno_firma_contrato,anno_cargue_secop,orden,entidad_centralizada,nombre_entidad,nit_entidad,codigo_entidad,modalidad_de_contratacion,estado_contrato,justificacion_modalidad_de,objeto_del_contrato,descripcion_del_proceso,tipo_de_contrato,municipio_entidad,departamento_entidad,id_grupo,nombre_grupo,id_familia,nombre_familia,id_clase,nombre_clase,tipo_doc_contratista,identificacion_contratista,nom_razon_social_contratista,tipo_doc_representante_legal,identificacion_representante_legal,nombre_representante_legal,genero_representante_legal,fecha_de_firma_del_contrato,fecha_de_inicio_del_contrato,plazo_de_ejec_del_contrato,rango_de_ejec_del_contrato,tiempo_adiciones_en_dias,tiempo_adiciones_en_meses,fecha_de_fin_del_contrato,valor_del_contrato,valor_total_de_adiciones,valor_contrato_con_adiciones,moneda,fecha_liquidacion,Tipo_servicio,codigo_de_categoria_principal,Database,proceso_de_compra,id_contrato,referencia_del_contrato,sector,rama,localizaci_n,nacionalidad_representante_legal,es_grupo,es_pyme,fecha_de_inicio_de_ejecucion,fecha_de_fin_de_ejecucion,fecha_fin_liquidacion
71,21-4-12044316-11157010,21-4-12044316,CFLSPCMC00202021,20,2021,2021,Territorial,DISTRITO CAPITAL,BOGOTÁ DC IED FABIO LOZANO SIMONELLI,No Definido,01002071,Contratación régimen especial,Celebrado,"Equipos y Suministros de Defensa, Orden Public...",SEGUN LOS APROBADO EN EL CONSEJO DIRECTIVO DE ...,SEGUN LOS APROBADO EN EL CONSEJO DIRECTIVO DE ...,Suministros,Bogotá DC,Bogotá DC,E,[E] Productos de Uso Final,4610,Armas ligeras y munición,461017,Sistemas de manipulación de munición,Nit de Persona Jurídica,900387129,SOLUCIONES ISSO SAS,Cédula de Ciudadanía,79523229,FRANCISCO JAVIER GAMBOA PEDRAZA,Hombre,2021-06-11T00:00:00.000,2021-06-11T00:00:00.000,30,D,0,0,2021-07-11T00:00:00.000,993650,0,993650,Peso Colombiano,NaN,Suministro de Armas,V1.46101700,SECOP_I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,18-13-7889356-7195387,18-13-7889356,IPMCSG0172018,017,2018,2018,Territorial,DEPARTAMENTAL DESCENTRALIZADO,GUAINÍA GOBERNACIÓN,No Definido,294000001,Contratación Mínima Cuantía,Liquidado,"Equipos y Suministros de Defensa, Orden Public...",SUMINISTRO Y MANTENIMIENTO DE LAS INSTALACIONE...,SUMINISTRO Y MANTENIMIENTO DE LAS INSTALACIONE...,Suministros,Inírida,Guainía,E,[E] Productos de Uso Final,4610,Armas ligeras y munición,461017,Sistemas de manipulación de munición,Nit de Persona Natural,19002620,FRANCISCO IVAN AGUDELO OSPINA,Cédula de Ciudadanía,19002620,FRANCISCO IVAN AGUDELO OSPINA,No Definido,2018-02-23T00:00:00.000,2018-02-27T00:00:00.000,10,D,0,0,2018-03-09T00:00:00.000,9792777,0,9792777,Peso Colombiano,2018-06-29T00:00:00.000,Suministro de Armas,V1.46101700,SECOP_I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,18-13-8026374-7307874,18-13-8026374,IP No0232018,CV No0062018,2018,2018,Territorial,DISTRITAL MUNICIPAL NIVEL 6,CUNDINAMARCA ALCALDÍA MUNICIPIO DE TIBACUY,800097176-6,225805011,Contratación Mínima Cuantía,Liquidado,"Equipos y Suministros de Defensa, Orden Public...",ADQUISICION DE PISTOLA NOQUEADORA DE GANADO CA...,ADQUISICION DE PISTOLA NOQUEADORA DE GANADO CA...,Compraventa,Tibacuy,Cundinamarca,E,[E] Productos de Uso Final,4610,Armas ligeras y munición,461015,Armas de fuego,Nit de Persona Jurídica,900481210,SOCIEDAD AGROPECOL DE COLOMBIA SAS,Cédula de Ciudadanía,2965727,CARLOS GIOVANNY GUTIERREZ CHARRY,No Definido,2018-04-23T00:00:00.000,2018-06-19T00:00:00.000,20,D,0,0,2018-07-09T00:00:00.000,9750000,0,9750000,Peso Colombiano,2018-08-13T00:00:00.000,Suministro de Armas,V1.46101500,SECOP_I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
650,18-4-7610075-6921291,18-4-7610075,1032017,10922017,2017,2018,Nacional,CENTRALIZADO,INDUMIL INDUSTRIA MILITAR,899999044,115014003,Contratación régimen especial,Liquidado,"Equipos y Suministros de Defensa, Orden Public...",Adquisición munición de escopeta deportiva var...,Adquisición munición de escopeta deportiva var...,Compraventa,Bogotá DC,Bog

In [165]:
Armas_df['valor_del_contrato'] = Armas_df['valor_del_contrato'].astype(int)
Armas_df['valor_del_contrato'].sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


441592903596

In [167]:
ls

 Alternativas.zip
'BD APO Personal'/
'BD DANE Encuesta Seguridad'/
'BD DANE Poblacion y Area de Municipios'/
'BD POLICIA - Delitos de impacto - 2009-2021'/
'BD RENOVA - Contratos cliente gobierno'/
'BD RENOVA - Contratos Todos los Clientes'/
'BD RENOVA - Delitos'/
'BD RENOVA - Empresas'/
'BD SECOP-II - Contratos'/
'Copy of fasecolda_dataset_4749202.csv'
 CU_Delitos/
 CU_Gasto/
'Documentos de Trabajo Proyecto Observatorio_12-22-2021.zip'
 DTS-Diagnoìstico.zip
 fasecolda_dataset_4749202.csv
 SWEETVIZ_REPORT.html
 temp.csv


In [168]:
Armas_df.to_csv('CU_Gasto/Armas_df.csv')

# Step 3: Ingeniería de Atributos

In [105]:
# Eliminar contratos que no serán tomados en cuenta

In [107]:
SECOP_df.drop(SECOP_df[SECOP_df['codigo_de_categoria_principal'].str.contains('V1.461519')].index, inplace = True)
SECOP_df.drop(SECOP_df[SECOP_df['codigo_de_categoria_principal'].str.contains('V1.921219')].index, inplace = True)

In [108]:
SECOP_df.drop(SECOP_df[SECOP_df['Tipo_servicio']=='Suministro de Armas'].index, inplace = True)

In [109]:
SECOP_df.loc[SECOP_df['uid'].isna(), 'uid'] = SECOP_df['id_contrato']


In [110]:
SECOP_df.drop(['localizaci_n',
               'id_contrato',
               'id_grupo', 
               'nombre_grupo',
               'id_familia', 
               'nombre_familia', 
               'fecha_liquidacion', 
               'fecha_fin_liquidacion', 
               'fecha_de_inicio_de_ejecucion',
               'fecha_de_fin_de_ejecucion'], axis=1, inplace=True)

In [135]:
contratos_SECOP_df = SECOP_df.copy()
contratos_SECOP_df.shape

(30896, 49)

## Variable Temporal Año-Mes

In [136]:
# Convertir la variable 'Fecha_de_firma" a tipo Datetime y extrae el año y el mes
#contratos_SECOP_df['Year-Month'] = contratos_SECOP_df['year'] +  contratos_SECOP_df['month']
#contratos_SECOP_df['Year-Month'] = pd.DatetimeIndex(contratos_SECOP_df['fecha_de_firma']).year
contratos_SECOP_df['Anio-Mes'] = pd.to_datetime(contratos_SECOP_df['fecha_de_firma_del_contrato']).dt.to_period("M")

In [137]:
contratos_SECOP_df.loc[contratos_SECOP_df['anno_cargue_secop']=='N/D', 'anno_cargue_secop'] = contratos_SECOP_df['anno_firma_contrato']

## Tipo de Datos

In [138]:
columnas = [
         'fecha_de_firma_del_contrato',
         'fecha_de_inicio_del_contrato', 
         'fecha_de_fin_del_contrato', 
]
for column in columnas:
    try:
        contratos_SECOP_df[column] = pd.to_datetime(contratos_SECOP_df[column])
    except:
        print(column)


In [139]:
nombres=[
         'orden', 
         'entidad_centralizada',
         'modalidad_de_contratacion', 
         'estado_contrato',
         'justificacion_modalidad_de', 
         'objeto_del_contrato',
         'tipo_de_contrato', 
         'municipio_entidad',
         'departamento_entidad', 
         'id_clase',
         'codigo_de_categoria_principal', 
         'tipo_doc_contratista', 
         'tipo_doc_representante_legal',
         'rango_de_ejec_del_contrato', 
         'Tipo_servicio',
         'Database',
         'moneda',
         'sector', 
         'rama', 
         'nacionalidad_representante_legal', 
         'es_grupo', 
         'es_pyme'
]
for column in nombres:
    try:
        contratos_SECOP_df[column] = contratos_SECOP_df[column].astype('category')
    except:
        print(column)

In [140]:
nombres=[
         'nombre_entidad', 
         'nit_entidad', 
         'codigo_entidad',
         'descripcion_del_proceso', 
         'nombre_clase',
         'identificacion_contratista',
         'nom_razon_social_contratista', 
         'identificacion_representante_legal', 
         'nombre_representante_legal',
         'genero_representante_legal' 
]
for column in nombres:
    try:
        contratos_SECOP_df[column] = contratos_SECOP_df[column].astype('string')
    except:
        print(column)

In [141]:
nombres=['anno_firma_contrato',
         'anno_cargue_secop',
         'plazo_de_ejec_del_contrato',
]
for column in nombres:
    try:
        contratos_SECOP_df[column] = contratos_SECOP_df[column].astype(int)
    except:
        print(column)

In [142]:
convert_dict = {
                'valor_del_contrato':           'float64',
                'valor_total_de_adiciones':     'float64', 
                'valor_contrato_con_adiciones': 'float64',
                'tiempo_adiciones_en_dias': 'float64', # .astype(float).astype('Int64')
                'tiempo_adiciones_en_meses': 'float64', # pd.to_numeric(df[col],errors='coerce').astype(pd.Int64Dtype())
                #'plazo_de_ejec_del_contrato': "float64"
                }
contratos_SECOP_df = contratos_SECOP_df.astype(convert_dict)

In [143]:
contratos_SECOP_df.dtypes

uid                                           object
numero_de_constancia                          object
numero_de_proceso                             object
numero_de_contrato                            object
anno_firma_contrato                            int64
anno_cargue_secop                              int64
orden                                       category
entidad_centralizada                        category
nombre_entidad                                string
nit_entidad                                   string
codigo_entidad                                string
modalidad_de_contratacion                   category
estado_contrato                             category
justificacion_modalidad_de                  category
objeto_del_contrato                         category
descripcion_del_proceso                       string
tipo_de_contrato                            category
municipio_entidad                           category
departamento_entidad                        ca

In [144]:
contratos_SECOP_df.select_dtypes(include='object').columns

Index(['uid', 'numero_de_constancia', 'numero_de_proceso',
       'numero_de_contrato', 'proceso_de_compra', 'referencia_del_contrato'],
      dtype='object')

In [145]:
contratos_SECOP_df.shape

(30896, 50)

## Analisis de duplicados

In [146]:
contratos_SECOP_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30896 entries, 0 to 31615
Data columns (total 50 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   uid                                 30896 non-null  object        
 1   numero_de_constancia                27018 non-null  object        
 2   numero_de_proceso                   27018 non-null  object        
 3   numero_de_contrato                  27013 non-null  object        
 4   anno_firma_contrato                 30896 non-null  int64         
 5   anno_cargue_secop                   30896 non-null  int64         
 6   orden                               30896 non-null  category      
 7   entidad_centralizada                30896 non-null  category      
 8   nombre_entidad                      30896 non-null  string        
 9   nit_entidad                         30896 non-null  string        
 10  codigo_entidad        

In [123]:
#contratos_SECOP_df = contratos_SECOP_df.sort_values(by = 'valor_del_contrato', ascending = False)

In [148]:
%%time
contratos_SECOP_df = contratos_SECOP_df.drop_duplicates(subset = ['numero_de_constancia', 'numero_de_proceso',
       'numero_de_contrato', 'anno_firma_contrato', 'anno_cargue_secop',  
       'nit_entidad', 'identificacion_contratista', 'identificacion_representante_legal', 'fecha_de_firma_del_contrato',
       'proceso_de_compra', 'referencia_del_contrato'], keep='first')   #2 duplicados

CPU times: user 140 ms, sys: 1.75 ms, total: 141 ms
Wall time: 141 ms


In [149]:
contratos_SECOP_df.shape

(30835, 50)

## Valores Nominales de los Contratos

In [126]:
contratos_SECOP_df['valor_contrato_con_adiciones'].describe()

count            26,957.00
mean        334,586,464.54
std       3,205,843,998.63
min                   0.00
25%           2,300,000.00
50%           7,316,640.00
75%          56,001,065.00
max     204,304,137,771.00
Name: valor_contrato_con_adiciones, dtype: float64

In [127]:
#Esto no funciona:
#contratos_SECOP_df.loc[contratos_SECOP_df['valor_contrato_con_adiciones'].isna(), 'valor_contrato_con_adiciones'] = contratos_SECOP_df['valor_del_contrato'] + contratos_SECOP_df['valor_total_de_adiciones']
#contratos_SECOP_df['valor_contrato_con_adiciones'].describe()

count            26,957.00
mean        334,586,464.54
std       3,205,843,998.63
min                   0.00
25%           2,300,000.00
50%           7,316,640.00
75%          56,001,065.00
max     204,304,137,771.00
Name: valor_contrato_con_adiciones, dtype: float64

In [128]:
contratos_SECOP_df.loc[contratos_SECOP_df['valor_contrato_con_adiciones'] < 10000].shape

(71, 50)

In [154]:
# Calcular el valor real de los contratos ajustando por inflación

IPC_Dict = {
    2000 : 100,
    2001 : 	107.97,
    2002 : 	114.83,
    2003 : 	123.02,
    2004 : 	130.28,
    2005 : 	136.86,
    2006 : 	142.73,
    2007 : 	150.65,
    2008 : 	161.19,
    2009 : 	167.96,
    2010 : 	171.77,
    2011 : 	177.65,
    2012 : 	183.27,
    2013 : 	186.97,
    2014 : 	192.39,
    2015 : 	201.99,
    2016 : 	217.17,
    2017 : 	226.53,
    2018 : 	233.87,
    2019 : 	242.12,
    2020 : 	248.23,
    2021 : 	256.91,
    2022 : 	277.89,
}

Inflation_Dict = {
    2000 : 0.6,
    2001 : 	0.6,
    2002 : 	0.6,
    2003 : 	0.6,
    2004 : 	0.6,
    2005 : 	0.6,
    2006 : 	0.61,
    2007 : 	0.62,
    2008 : 	0.63,
    2009 : 	0.66,
    2010 : 	0.69,
    2011 : 	0.71,
    2012 : 	0.73,
    2013 : 	0.79,
    2014 : 	0.82,
    2015 : 	0.85,
    2016 : 	0.90,
    2017 : 	0.94,
    2018 : 	0.95,
    2019 : 	0.98,
    2020 : 	1,
    2021 : 	1.05,
    2022 : 	1.08,
}


#IPC_Dict[2003]

#Se calcula el valor del contrato en pesos en 2015

def valor_2015(row):

    if (row['anno_firma_contrato']>0):
        if (row['valor_contrato_con_adiciones']>0):
            Value  =  row['valor_contrato_con_adiciones'] * 201.99/IPC_Dict[row['anno_firma_contrato']]
            return Value
        elif (row['valor_del_contrato']>0):
            Value  =  row['valor_del_contrato'] * 201.99/IPC_Dict[row['anno_firma_contrato']]
            return Value

def valor_2000(row):
    if (row['anno_firma_contrato']>0):
        if (row['valor_contrato_con_adiciones']>0):
            Value  =  row['valor_contrato_con_adiciones'] * 248.23/IPC_Dict[row['anno_firma_contrato']]
            Value  =  Value / Inflation_Dict[row['anno_firma_contrato']]
            return Value
        elif (row['valor_del_contrato']>0):
            Value  =  row['valor_del_contrato'] * 248.23/IPC_Dict[row['anno_firma_contrato']]
            Value  =  Value / Inflation_Dict[row['anno_firma_contrato']]
            return Value
    
contratos_SECOP_df['Valor_Contrato_real_2015'] = contratos_SECOP_df.apply(lambda row: valor_2015(row), axis=1)
contratos_SECOP_df['Valor_Contrato_Presente_2020'] = contratos_SECOP_df.apply(lambda row: valor_2000(row), axis=1)
contratos_SECOP_df['Valor_Contrato_Presente_2020'].describe()


# Código de referencia usado en versiones anteriores
'''
def valor_2015(row):
    if (row['MONTO_CONTRATO']>0) & (row['Año_Inicio']>0):
        Value  =  row['MONTO_CONTRATO'] * 201.99 / IPC_Dict[row['Año_Inicio']]
        return Value

def valor_presente(row):
    if (row['MONTO_CONTRATO']>0) & (row['Año_Inicio']>0):
        Value  =  row['MONTO_CONTRATO'] * 277.89 / IPC_Dict[row['Año_Inicio']]
        return Value
    
df2['Valor_Contrato_nominal_2015'] = df2.apply(lambda row: valor_2015(row), axis=1)
df2['Valor_Contrato_Presente_2022'] = df2.apply(lambda row: valor_presente(row), axis=1)
df2['Valor_Contrato_Presente_2022'].describe()'''

"\ndef valor_2015(row):\n    if (row['MONTO_CONTRATO']>0) & (row['Año_Inicio']>0):\n        Value  =  row['MONTO_CONTRATO'] * 201.99 / IPC_Dict[row['Año_Inicio']]\n        return Value\n\ndef valor_presente(row):\n    if (row['MONTO_CONTRATO']>0) & (row['Año_Inicio']>0):\n        Value  =  row['MONTO_CONTRATO'] * 277.89 / IPC_Dict[row['Año_Inicio']]\n        return Value\n    \ndf2['Valor_Contrato_nominal_2015'] = df2.apply(lambda row: valor_2015(row), axis=1)\ndf2['Valor_Contrato_Presente_2022'] = df2.apply(lambda row: valor_presente(row), axis=1)\ndf2['Valor_Contrato_Presente_2022'].describe()"

In [155]:
contratos_SECOP_df.loc[contratos_SECOP_df['Valor_Contrato_Presente_2020'].isna(), 'valor_del_contrato'].value_counts().head()

0.00    95
Name: valor_del_contrato, dtype: int64

In [156]:
# Normalizacion de los valores de los contratos corrigiendo por duracion y cantidad de puestos contratados?

# Calular el valor real de los contratos ajustando por duración
def valor_diario(row):
    if (row['plazo_de_ejec_del_contrato']>0) & (row['Valor_Contrato_Presente_2020']>0):
        return  (row['Valor_Contrato_Presente_2020'] / row['plazo_de_ejec_del_contrato'])
    else:
        return 0

contratos_SECOP_df['Valor_diario_Contrato'] = contratos_SECOP_df.apply(lambda row: valor_diario(row), axis=1)
contratos_SECOP_df['Valor_diario_Contrato'].describe()

count          30,835.00
mean        3,718,289.00
std        32,756,841.56
min                 0.00
25%            38,857.91
50%           280,951.81
75%         1,253,836.76
max     3,114,036,111.53
Name: Valor_diario_Contrato, dtype: float64

## Guardar el Archivo

In [33]:
#contratos_SECOP_df.to_csv('/content/drive/MyDrive/Observatorio SPVS/BD SECOP-II - Contratos/SECOP-I-II-Completo.csv', index = False, header=True)

In [13]:
#contratos_SECOP_df = pd.read_csv('/content/drive/MyDrive/Observatorio SPVS/BD SECOP-II - Contratos/SECOP-I-II-Completo.csv')
contratos_SECOP_df.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (42,43,44,45,46,47,48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(30835, 53)

In [159]:
contratos_SECOP_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30835 entries, 0 to 31615
Data columns (total 53 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   uid                                 30835 non-null  object        
 1   numero_de_constancia                26957 non-null  object        
 2   numero_de_proceso                   26957 non-null  object        
 3   numero_de_contrato                  26952 non-null  object        
 4   anno_firma_contrato                 30835 non-null  int64         
 5   anno_cargue_secop                   30835 non-null  int64         
 6   orden                               30835 non-null  category      
 7   entidad_centralizada                30835 non-null  category      
 8   nombre_entidad                      30835 non-null  string        
 9   nit_entidad                         30835 non-null  string        
 10  codigo_entidad        